In [1]:
%matplotlib inline
import pandas as pd
#import pandas.util.testing as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns
from os import walk
from tqdm import tqdm

In [2]:
import pysam

In [3]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01952.HIFIRW.ONTUL.na.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0010036.HG01952:1483108-1483621"))

In [4]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928:8920188-8920701"))

In [5]:
HG00358 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG00358/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
HC02666 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HC02666/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
HG01890 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01890/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
HG01952 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01952/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
HG01106 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01106/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
HG02011 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG02011/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])
T2T = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/T2T/BlockSize_dataframe.csv").drop(columns=['Unnamed: 0'])

## Spacer Alus

In [97]:
def findSpacerAlus(coordinateDF, file):
    newDF = coordinateDF[coordinateDF['Block_Type']=='Spacer Block'].copy()
    
    with open(file) as f:
        Lines = f.readlines()

        for line in tqdm(Lines): 

            goodLine = ' '.join(line.split())

            if 'SINE/Alu' in goodLine:
                for row in newDF.index:
                    if int(goodLine.split(" ")[6])>= int(newDF.at[row,'Start']) and int(goodLine.split(" ")[7])<= int(newDF.at[row,'End']):
                        print("SPACER BLOCK: "+str(newDF.at[row,'Block_Number']))
                        print("SPACER COORDINATES: "+str(newDF.at[row,'Start'])+"-"+str(newDF.at[row,'End']))
                        print(goodLine+'\n')

                    else:
                        continue
    f.close()

In [107]:
def findSpacerAlus_T2T(coordinateDF, file):
    newDF = coordinateDF[coordinateDF['Block_Type']=='Spacer Block'].copy()
    
    with open(file) as f:
        Lines = f.readlines()

        for line in tqdm(Lines): 

            goodLine = ' '.join(line.split())

            if 'SINE/Alu' in goodLine:
                for row in newDF.index:
                    if int(goodLine.split(" ")[5])>= int(newDF.at[row,'Start']) and int(goodLine.split(" ")[6])<= int(newDF.at[row,'End']):
                        print("SPACER BLOCK: "+str(newDF.at[row,'Block_Number']))
                        print("SPACER COORDINATES: "+str(newDF.at[row,'Start'])+"-"+str(newDF.at[row,'End']))
                        print(goodLine+'\n')

                    else:
                        continue
    f.close()

In [100]:
findSpacerAlus(HC02666,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HC02666_finalized.txt' )

100%|██████████████████████████████████| 22892/22892 [00:00<00:00, 45177.18it/s]


In [101]:
findSpacerAlus(HG00358,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HG00358_finalized.txt' )

100%|██████████████████████████████████| 27925/27925 [00:01<00:00, 23862.64it/s]


In [102]:
findSpacerAlus(HG01890,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HG01890_finalized.txt' )

100%|██████████████████████████████████| 16757/16757 [00:00<00:00, 54802.81it/s]


In [103]:
findSpacerAlus(HG01952,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HG01952_finalized.txt' )

 35%|████████████▎                      | 7241/20665 [00:00<00:00, 72368.83it/s]

SPACER BLOCK: 0
SPACER COORDINATES: 619972-1881157
45248 2931 1.9 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0010036.HG01952 1483208 1483521 21997344 True AluYb8 SINE/Alu 4 314 1 34660 False



100%|██████████████████████████████████| 20665/20665 [00:00<00:00, 37233.64it/s]


In [104]:
findSpacerAlus(HG01106,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HG01106_finalized.txt' )

100%|██████████████████████████████████| 32711/32711 [00:01<00:00, 22023.91it/s]


In [105]:
findSpacerAlus(HG02011,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/HG02011_finalized.txt' )

100%|██████████████████████████████████| 19741/19741 [00:00<00:00, 36459.34it/s]


In [108]:
findSpacerAlus_T2T(T2T,'/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/finalized/T2T.ChrY.Final.fasta.out' )

100%|██████████████████████████████████| 29612/29612 [00:01<00:00, 20820.87it/s]


# Fragmented Genomes

In [126]:
def findSpacerAlus_FragmentedGenomes(DF, file):
    OrientationDict = {'True':'C', 'False':'+'}
    AluList = [str(x) for x in DF.index]
    
    with open(file) as f:
        Lines = f.readlines()

        for line in tqdm(Lines): 

            goodLine = ' '.join(line.split())

            if 'SINE/Alu' in goodLine:
                #print(goodLine)
                #break
                if (str(OrientationDict[goodLine.split(" ")[9]])+"_"+str(goodLine.split(" ")[10])+"_"+str(goodLine.split(" ")[6])+"-"+str(goodLine.split(" ")[7])) in AluList:
                    continue
                else:
                    print(str(OrientationDict[goodLine.split(" ")[9]])+"_"+str(goodLine.split(" ")[10])+"_"+str(goodLine.split(" ")[6])+"-"+str(goodLine.split(" ")[7]))
                    print(goodLine+"\n")
                
    f.close()

In [61]:
NA19317_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19317/NA19317_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19317_piece2 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19317/NA19317_piece2_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19347_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19347/NA19347_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19347_piece2 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19347/NA19347_piece2_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
HG01928_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/HG01928/HG01928_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
HG01928_piece2 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/HG01928/HG01928_piece2_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19705_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19705/NA19705_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19705_piece2 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19705/NA19705_piece2_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')

In [129]:
findSpacerAlus_FragmentedGenomes(NA19317_piece1, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19317_part1_finalized.tsv')

100%|█████████████████████████████████| 15928/15928 [00:00<00:00, 224172.86it/s]


In [130]:
findSpacerAlus_FragmentedGenomes(NA19317_piece2, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19317_part2_finalized.tsv')

100%|███████████████████████████████████| 3764/3764 [00:00<00:00, 352687.72it/s]


In [131]:
findSpacerAlus_FragmentedGenomes(NA19347_piece1, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19347_part1_finalized.tsv')

100%|█████████████████████████████████| 10421/10421 [00:00<00:00, 400847.77it/s]


In [132]:
findSpacerAlus_FragmentedGenomes(NA19347_piece2, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19347_part2_finalized.tsv')

100%|███████████████████████████████████| 3225/3225 [00:00<00:00, 321603.20it/s]


In [133]:
findSpacerAlus_FragmentedGenomes(HG01928_piece1, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/HG01928_part1_finalized.tsv')

100%|███████████████████████████████████| 8233/8233 [00:00<00:00, 327314.74it/s]

C_AluYb8_8920288-8920601
41784 2931 1.9 0.0 0.0 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 8920288 8920601 8266721 True AluYb8 SINE/Alu 4 314 1 13008 False



In [134]:
findSpacerAlus_FragmentedGenomes(HG01928_piece2, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/HG01928_part2_finalized.tsv')

100%|█████████████████████████████████| 12907/12907 [00:00<00:00, 186297.72it/s]


In [135]:
findSpacerAlus_FragmentedGenomes(NA19705_piece1, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19705_part1_finalized.tsv')

100%|█████████████████████████████████| 18054/18054 [00:00<00:00, 188323.57it/s]


In [137]:
findSpacerAlus_FragmentedGenomes(NA19705_piece2, '/home/mark/Desktop/chromosome_y/assemblies/repeatmasker/postMay232022/Fragmented/Finalized/NA19705_part2_finalized.tsv')

100%|███████████████████████████████████| 3702/3702 [00:00<00:00, 337631.03it/s]


# Two Closely Related Genomes breakpoints

In [ ]:
#Piece 1
#RB 0
#RB 1-2

In [26]:
NA19317_piece1[NA19317_piece1['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_29143532-29143823,0,860,['44594 1366 19.9 0.0 5.4 chrY.01-23.01.PAR1-H...,#073B4C,<999
C_AluY_29145954-29146242,0,2411,['44596 414 24.2 3.9 3.7 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29148369-29148660,0,2411,['44599 410 24.2 3.9 3.8 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29150778-29151065,0,2402,['44602 376 27.0 2.9 3.9 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29153190-29153478,0,2402,['44605 354 20.9 8.6 3.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29155606-29155896,0,2411,['44608 386 18.8 9.2 2.7 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29158021-29158308,0,2409,['44611 425 23.8 3.7 3.9 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_29160437-29160699,0,3404,['44614 295 27.2 5.8 3.0 chrY.01-23.01.PAR1-HE...,#FFD166,3000-3999
+_AluY_29162867-29163158,0,2399,['44617 3936 10.1 0.0 0.0 chrY.01-23.01.PAR1-H...,#06D6A0,2000-2999


In [30]:
NA19317_piece1.loc['C_AluY_29158021-29158308']['Group']

"['44611 425 23.8 3.7 3.9 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29156471 29158020 17922702 False (TATATAA)n Simple_repeat 1 1547 0 43966 False', '44612 1345 19.8 0.0 5.5 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29158021 29158308 17922414 True AluY SINE/Alu 12 289 17 43967 False', '44613 4051 9.3 0.0 0.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29158312 29158879 17921843 True HSATI Satellite 0 568 1 43968 False']"

In [33]:
29158888-29158879

9

In [28]:
NA19317_piece1.loc['C_AluY_29160437-29160699']['Group']

"['44614 295 27.2 5.8 3.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29158888 29160434 17920288 False (ATATATC)n Simple_repeat 1 1588 0 43969 False', '44615 1226 20.6 0.0 6.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29160437 29160699 17920023 True AluY SINE/Alu 12 289 42 43970 False', '44616 422 24.1 3.9 3.5 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29160741 29162291 17918431 False (ATTATAT)n Simple_repeat 1 1556 0 43971 False']"

In [31]:
29162305-29162291

14

In [29]:
NA19317_piece1.loc['+_AluY_29162867-29163158']['Group']

"['44617 3936 10.1 0.0 0.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29162305 29162867 17917855 False HSATI Satellite 6 568 0 43972 False', '44618 1403 19.2 0.0 5.4 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29162867 29163158 17917564 False AluY SINE/Alu 13 289 12 43973 True', '44619 388 19.0 8.9 2.9 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29163160 29164703 17916019 False (TA)n Simple_repeat 1 1634 0 43974 False']"

In [44]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19317.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317:29158888-29162305"))



In [62]:
NA19317_piece1[NA19317_piece1['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_29933645-29933936,1,1845,['45172 1414 19.9 0.0 5.4 chrY.01-23.01.PAR1-H...,#118AB2,1000-1999
+_AluY_29936064-29936355,1,2413,['45174 3920 10.2 0.0 0.0 chrY.01-23.01.PAR1-H...,#06D6A0,2000-2999
+_AluY_29938484-29938774,1,2412,['45177 4008 9.5 0.0 0.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
+_AluY_29940903-29941194,1,2413,['45180 3990 9.7 0.0 0.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
+_AluY_29943323-29943614,1,2424,['45183 3932 10.0 0.0 0.0 chrY.01-23.01.PAR1-H...,#06D6A0,2000-2999
+_AluY_29945754-29946045,1,2411,['45186 3949 9.9 0.0 0.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
+_AluY_29948172-29948463,1,2413,['45189 3983 9.9 0.0 0.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
+_AluY_29953000-29953291,1,2411,['45197 3926 10.4 0.0 0.0 chrY.01-23.01.PAR1-H...,#06D6A0,2000-2999
+_AluY_29955417-29955708,1,2412,['45200 3890 10.2 0.2 0.0 chrY.01-23.01.PAR1-H...,#06D6A0,2000-2999


In [63]:
NA19317_piece1[NA19317_piece1['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_30067027-30067318,2,859,['45247 1327 21.0 0.0 5.4 chrY.01-23.01.PAR1-H...,#073B4C,<999
C_AluY_30069443-30069735,2,2410,['45249 418 24.5 3.5 3.8 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30071858-30072149,2,2407,['45252 418 24.0 4.0 3.6 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30074276-30074567,2,2364,['45255 394 25.4 2.9 4.5 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30076633-30076924,2,2401,['45258 410 24.2 3.5 4.1 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30079056-30079345,2,2414,['45261 405 24.7 4.0 3.5 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30081474-30081765,2,2413,['45264 354 20.3 9.0 3.0 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30083889-30084180,2,2408,['45267 423 24.5 3.5 3.6 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999
C_AluY_30086304-30086595,2,2408,['45270 408 24.1 4.1 3.7 chrY.01-23.01.PAR1-HE...,#06D6A0,2000-2999


In [64]:
NA19317_piece1.loc['+_AluY_29979587-29979878']['Group']

"['45232 3873 10.9 0.0 0.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29979020 29979587 17101135 False HSATI Satellite 1 568 0 44525 False', '45233 1470 18.1 0.0 5.4 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29979587 29979878 17100844 False AluY SINE/Alu 13 289 12 44526 True', '45234 383 25.8 3.1 4.4 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29979881 29979916 17100806 False (ATATATA)n Simple_repeat 1 1533 0 44527 False', '45235 388 18.5 9.1 3.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 29979917 29981432 17099290 False (TA)n Simple_repeat 1 1606 0 44528 False']"

In [65]:
NA19317_piece1.loc['C_AluY_30067027-30067318']['Group']

"['45247 1327 21.0 0.0 5.4 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 30067027 30067318 17013404 True AluY SINE/Alu 12 289 13 44540 True', '45248 3940 10.4 0.0 0.0 chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317 30067318 30067885 17012837 True HSATI Satellite 0 568 1 44541 False']"

In [68]:
30067318-1600

30065718

In [71]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19317.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317:29979587-29981432"))


In [70]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19317.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317:30065718-30067318"))


In [ ]:
#############################################################################################33

In [53]:
NA19347_piece1[NA19347_piece1['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_18893409-18893700,1,1845,['44897 1414 19.9 0.0 5.4 chrY.09-23.01.HET1_c...,#118AB2,1000-1999
+_AluY_18895828-18896119,1,2413,['44899 3920 10.2 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_18898248-18898538,1,2412,['44902 4008 9.5 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_18900667-18900958,1,2413,['44905 3990 9.7 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_18903087-18903378,1,2424,['44908 3932 10.0 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_18905518-18905809,1,2411,['44911 3949 9.9 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_18907936-18908227,1,2411,['44914 3983 9.9 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_18910354-18910645,1,2411,['44917 4046 9.3 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_18912772-18913063,1,2411,['44920 3926 10.4 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999


In [54]:
NA19347_piece1[NA19347_piece1['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_19037468-19037759,2,859,['45050 1327 21.0 0.0 5.4 chrY.09-23.01.HET1_c...,#073B4C,<999
C_AluY_19039884-19040176,2,2410,['45052 418 24.5 3.5 3.8 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19042299-19042590,2,2407,['45055 418 24.0 4.0 3.6 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19044717-19045008,2,2364,['45058 394 25.4 2.9 4.5 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19047074-19047365,2,2401,['45061 410 24.2 3.5 4.1 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19049497-19049786,2,2414,['45064 405 24.7 4.0 3.5 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19051915-19052206,2,2413,['45067 354 20.3 9.0 3.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19054330-19054621,2,2408,['45070 423 24.5 3.5 3.6 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19056745-19057036,2,2408,['45073 408 24.1 4.1 3.7 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999


In [ ]:
#Breakpoint 2

In [55]:
NA19347_piece1.loc['+_AluY_18939359-18939650']['Group']

"['44955 3871 10.9 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18938792 18939359 9278416 False HSATI Satellite 1 568 0 23876 False', '44956 1455 18.5 0.0 5.4 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18939359 18939650 9278125 False AluY SINE/Alu 13 289 12 23877 True', '44957 383 25.8 3.1 4.4 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18939653 18939688 9278087 False (ATATATA)n Simple_repeat 1 1533 0 23878 False', '44958 388 18.5 9.1 3.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18939689 18941204 9276571 False (TA)n Simple_repeat 1 1606 0 23879 False']"

In [61]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19347.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347:18939359-18941204"))



In [56]:
NA19347_piece1.loc['C_AluY_19037468-19037759']['Group']

"['45050 1327 21.0 0.0 5.4 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 19037468 19037759 9180016 True AluY SINE/Alu 12 289 13 23968 True', '45051 3940 10.4 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 19037759 19038326 9179449 True HSATI Satellite 0 568 1 23969 False']"

In [58]:
19037468-1606

19035862

In [60]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19347.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347:19035862-19037759"))



In [ ]:
# Breakpoint 1

In [35]:
NA19347_piece1.loc['C_AluY_18117795-18118082']['Group']

"['44276 425 23.8 3.7 3.9 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18116245 18117794 10099981 False (TATATAA)n Simple_repeat 1 1547 0 23239 False', '44277 1345 19.8 0.0 5.5 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18117795 18118082 10099693 True AluY SINE/Alu 12 289 17 23240 False', '44278 4051 9.3 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18118086 18118653 10099122 True HSATI Satellite 0 568 1 23241 False']"

In [41]:
18118662-18118653

9

In [37]:
NA19347_piece1.loc['C_AluY_18120211-18120473']['Group']

"['44279 295 27.2 5.8 3.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18118662 18120208 10097567 False (ATATATC)n Simple_repeat 1 1588 0 23242 False', '44280 1226 20.6 0.0 6.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18120211 18120473 10097302 True AluY SINE/Alu 12 289 42 23243 False', '44281 422 24.1 3.9 3.5 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18120515 18122065 10095710 False (ATTATAT)n Simple_repeat 1 1556 0 23244 False']"

In [40]:
18122079-18122065

14

In [39]:
NA19347_piece1.loc['+_AluY_18122641-18122932']['Group']

"['44282 3936 10.1 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18122079 18122641 10095134 False HSATI Satellite 6 568 0 23245 False', '44283 1403 19.2 0.0 5.4 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18122641 18122932 10094843 False AluY SINE/Alu 13 289 12 23246 True', '44284 388 19.0 8.9 2.9 chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347 18122934 18124477 10093298 False (TA)n Simple_repeat 1 1634 0 23247 False']"

In [43]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19347.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.RV.unassigned-0000258.NA19347:18118662-18122079"))



In [ ]:
HG01928_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/HG01928/HG01928_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
HG01928_piece2 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/HG01928/HG01928_piece2_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19705_piece1 = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/Fragmented/NA19705/NA19705_piece1_rbDF.csv").drop(columns=['Unnamed: 0']).set_index('Alu')
NA19705_piece2

# HG01928

In [67]:
HG01928_piece1[HG01928_piece1['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_9347694-9347985,0,860,['41920 1366 19.9 0.0 5.4 chrY.17-23.01.AMPL6-...,#073B4C,<999
C_AluY_9350116-9350404,0,2411,['41922 415 24.0 3.9 3.8 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9352531-9352822,0,2411,['41925 407 24.3 3.9 3.8 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9354940-9355227,0,2402,['41928 379 26.9 2.9 3.9 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9357351-9357642,0,2404,['41931 358 20.7 8.6 3.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9359760-9360047,0,2402,['41934 379 26.9 2.9 3.9 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9362174-9362462,0,2404,['41937 358 20.7 8.6 3.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9364590-9364880,0,2418,['41940 390 18.8 9.1 2.7 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_9367005-9367292,0,2409,['41943 393 19.1 8.8 2.9 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999


In [69]:
HG01928_piece1.loc['C_AluY_9369421-9369683']['Group']

"['41946 299 27.5 5.4 3.1 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9367872 9369418 7817904 False (ATATATC)n Simple_repeat 1 1581 0 13169 False', '41947 1226 20.6 0.0 6.0 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9369421 9369683 7817639 True AluY SINE/Alu 12 289 42 13170 False', '41948 419 24.2 3.9 3.5 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9369725 9371275 7816047 False (ATTATAT)n Simple_repeat 1 1556 0 13171 False']"

In [71]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928:9367872-9371275"))


In [72]:
HG01928_piece1[HG01928_piece1['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_9874400-9874691,1,1846,['42209 1403 20.3 0.0 5.4 chrY.17-23.01.AMPL6-...,#118AB2,1000-1999
+_AluY_9876820-9877111,1,2411,['42211 3989 9.7 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9879238-9879529,1,2412,['42214 4065 9.2 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9881657-9881948,1,2413,['42217 3968 9.9 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9884077-9884367,1,2412,['42220 3983 9.7 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9886496-9886787,1,2413,['42223 4012 9.5 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9888916-9889207,1,2424,['42226 3961 9.9 0.0 0.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
+_AluY_9891347-9891638,1,2411,['42229 3926 10.0 0.0 0.0 chrY.17-23.01.AMPL6-...,#06D6A0,2000-2999
+_AluY_9893765-9894056,1,2409,['42232 3959 10.0 0.0 0.0 chrY.17-23.01.AMPL6-...,#06D6A0,2000-2999


In [73]:
HG01928_piece1[HG01928_piece1['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_10119413-10119704,2,859,['42376 1341 20.3 0.0 5.4 chrY.17-23.01.AMPL6-...,#073B4C,<999
C_AluY_10121829-10122121,2,2410,['42378 419 24.2 3.6 3.8 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10124246-10124538,2,2410,['42381 419 24.2 3.6 3.8 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10126661-10126952,2,2407,['42384 416 24.1 4.0 3.6 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10129079-10129370,2,2364,['42387 389 25.5 2.9 4.5 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10131436-10131727,2,2401,['42390 408 24.3 3.5 4.1 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10133859-10134148,2,2414,['42393 404 24.9 3.9 3.4 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10136277-10136568,2,2413,['42396 354 20.3 9.0 3.0 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999
C_AluY_10138691-10138982,2,2407,['42399 418 24.6 3.5 3.6 chrY.17-23.01.AMPL6-H...,#06D6A0,2000-2999


In [74]:
HG01928_piece1.loc['+_AluY_9992922-9993213']['Group']

"['42359 3926 10.4 0.0 0.0 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9992355 9992922 7194400 False HSATI Satellite 1 568 0 13555 False', '42360 1420 19.2 0.0 5.4 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9992922 9993213 7194109 False AluY SINE/Alu 13 289 12 13556 True', '42361 381 25.9 3.1 4.4 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9993216 9993251 7194071 False (ATATATA)n Simple_repeat 1 1533 0 13557 False', '42362 383 18.6 9.1 3.0 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 9993252 9994767 7192555 False (TA)n Simple_repeat 1 1606 0 13558 False']"

In [78]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928:9992922-9994767"))


In [75]:
HG01928_piece1.loc['C_AluY_10119413-10119704']['Group']

"['42376 1341 20.3 0.0 5.4 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 10119413 10119704 7067618 True AluY SINE/Alu 12 289 13 13571 True', '42377 3938 10.4 0.0 0.0 chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928 10119704 10120271 7067051 True HSATI Satellite 0 568 1 13572 False']"

In [77]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928:10118413-10119704"))


>chrY.17-23.01.AMPL6-HET3_Yq.RV.unassigned-0005883.HG01928:10118413-10119704
AATGGAATGGAATAGAAAGGAATGGAATGAAATGGAATGGAAAGGATTCGAATGGAATGC
AATCGAATGGAATGGAATCGAACGGAATGGAATAAAATGGAAGAAAACTGGCAAGAAATG
GAATCGAAATGAATGGAGTGTTATGGAACGGACTCAAAAGGAATTGAATGTAATAGAATG
GAGTGGAGTGGACTCGAATATAATGGACTGGAATGGAATGAAATCACATGGAATGGGAAC
GAATGGAATGGAATGGAAAGGAATGGAATCGAATGGAAAGGAATCGAATGGAAGGGAATG
AAATTGAATGGACTATATATTTTGTTATATATACTTTTATAACATAATTTACATAAAATA
ACATAATGTAATATATATTATATTATTTCATATTACATAATATATATTACATATCACCCA
ATATAAGTAACATACATAATATTATAATATAGATTATATTATGTTATATTAAATATCATA
TATATCATGTATCATATATTCTATAATATATATTATATATTATATAATATGTATGTTGTC
TATTATGTGATATATATAACTATATATATAATTATATTATTATACATATTATACATATAA
CTAGATAAACATGTAATTCTACATATTTTCGTTTATAATATATAGAATTATATAATCATA
TATAATTATTTATATTCTATAAAAAATTATATCATTGTATAAATTATAATATATAAAAAT
TATAATATGTACTACAAATATATACATTATATATCATATATGATATAGTACCTTTGTTAT
ATATCATAATACATATAAATGTGTTTTATGTTATCTATAATTATCTAATTTCTTATATAA
GGTGTATAATATGTATCATATATTATATATGTTATGTAATATATATAGTATATATAAGAT
GACACAGG

In [ ]:
########## 3rd Breakpoint ###################################

In [108]:
HG01928_piece2[HG01928_piece2['RepeatBlock']==14]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_13612022-13612313,14,1609,['59955 181 25.0 4.0 4.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_13614444-13614734,14,2414,['59958 374 25.9 3.9 3.3 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13616865-13617156,14,2415,['59963 381 25.9 3.6 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13619287-13619577,14,2414,['59968 374 26.1 3.6 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13621708-13621999,14,2415,['59973 381 25.9 3.6 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13624128-13624419,14,2413,['59978 428 23.8 3.7 3.7 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13626544-13626835,14,2409,['59981 430 23.9 3.8 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13628968-13629256,14,2414,['59984 422 23.7 4.0 3.6 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_13631361-13631652,14,2394,['59987 416 24.2 3.3 4.1 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [110]:
HG01928_piece2.loc['C_AluY_13684687-13684976']['Group']

"['60059 370 19.9 9.0 2.9 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13683130 13684682 1223717 False (TA)n Simple_repeat 1 1646 0 30925 False', '60060 1449 18.2 0.0 5.5 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13684687 13684976 1223423 True AluY SINE/Alu 14 287 13 30926 True', '60061 4043 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13684976 13685543 1222856 True HSATI Satellite 0 568 1 30927 False']"

In [111]:
HG01928_piece2.loc['+_AluY_13686214-13686492']['Group']

"['60062 157 19.3 9.8 2.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13685544 13686210 1222189 False (AT)n Simple_repeat 1 713 0 30928 False', '60063 1353 19.0 0.0 5.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13686214 13686492 1221907 False AluY SINE/Alu 26 289 12 30929 False', '60064 378 26.7 2.8 4.1 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 13686494 13688046 1220353 False (ATATATA)n Simple_repeat 1 1533 0 30930 False']"

In [113]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928:13685544-13688046"))


In [114]:
################## 4th Breakpoint ########################

In [115]:
HG01928_piece2[HG01928_piece2['RepeatBlock']==15]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_14074432-14074723,15,2380,['60285 3930 10.4 0.0 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_14076819-14077110,15,3695,['60288 4016 9.3 0.0 0.2 chrY.23-24.01.HET3_Yq...,#FFD166,3000-3999
+_AluY_14080520-14080811,15,2407,['60291 4005 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_14082933-14083224,15,2410,['60294 3854 10.6 0.2 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_14085351-14085642,15,2413,['60297 3957 9.9 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_14087771-14088062,15,2413,['60300 4010 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_14090191-14090482,15,2415,['60305 4008 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_14092614-14092905,15,1751,['60308 3884 10.4 0.0 0.2 chrY.23-24.01.HET3_Y...,#118AB2,1000-1999
C_AluY_14095934-14096225,15,2409,['60312 409 24.8 3.5 3.8 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [116]:
HG01928_piece2.loc['+_AluY_14092614-14092905']['Group']

"['60308 3884 10.4 0.0 0.2 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14092046 14092614 815785 False HSATI Satellite 1 568 0 31159 False', '60309 1468 18.8 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14092614 14092905 815494 False AluY SINE/Alu 13 289 12 31160 True', '60310 229 18.7 8.1 3.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14092907 14093796 814603 False (TA)n Simple_repeat 1 928 0 31161 False']"

In [118]:
14094384-14093796

588

In [117]:
HG01928_piece2.loc['C_AluY_14095934-14096225']['Group']

"['60312 409 24.8 3.5 3.8 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14094384 14095933 812466 False (TATATAA)n Simple_repeat 1 1547 0 31163 False', '60313 1439 18.5 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14095934 14096225 812174 True AluY SINE/Alu 12 289 13 31164 True', '60314 3962 9.9 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928 14096225 14096792 811607 True HSATI Satellite 0 568 1 31165 False']"

In [120]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01928.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0008584.HG01928:14092614-14096225"))


# NA19705

In [68]:
NA19705_piece1[NA19705_piece1['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_18982357-18982648,0,860,['42589 1366 19.9 0.0 5.4 chrY.09-23.01.HET1_c...,#073B4C,<999
C_AluY_18984779-18985067,0,2411,['42591 414 24.2 3.9 3.7 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18987194-18987485,0,2411,['42594 410 24.2 3.9 3.8 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18989603-18989890,0,2402,['42597 376 27.0 2.9 3.9 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18992015-18992303,0,2402,['42600 354 20.9 8.6 3.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18994428-18994718,0,2408,['42603 386 18.9 9.2 2.7 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18996842-18997129,0,2408,['42606 424 23.7 3.7 3.9 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_18999258-18999520,0,3404,['42609 295 27.2 5.8 3.0 chrY.09-23.01.HET1_ce...,#FFD166,3000-3999
+_AluY_19001688-19001979,0,2399,['42612 3914 10.3 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999


In [79]:
NA19705_piece1.loc['C_AluY_18999258-18999520']['Group']

"['42609 295 27.2 5.8 3.0 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 18997709 18999255 19051507 False (ATATATC)n Simple_repeat 1 1588 0 38030 False', '42610 1226 20.6 0.0 6.0 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 18999258 18999520 19051242 True AluY SINE/Alu 12 289 42 38031 False', '42611 422 24.1 3.9 3.5 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 18999562 19001112 19049650 False (ATTATAT)n Simple_repeat 1 1556 0 38032 False']"

In [81]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19705.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705:18997709-19001112"))


In [82]:
NA19705_piece1[NA19705_piece1['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_19703236-19703527,1,1845,['43270 1414 19.9 0.0 5.4 chrY.09-23.01.HET1_c...,#118AB2,1000-1999
+_AluY_19705655-19705946,1,2413,['43272 3920 10.2 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_19708075-19708366,1,2411,['43275 3955 10.0 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_19710492-19710783,1,2412,['43278 3890 10.2 0.2 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_19712912-19713203,1,2408,['43281 3939 10.0 0.0 0.0 chrY.09-23.01.HET1_c...,#06D6A0,2000-2999
+_AluY_19715327-19715618,1,2408,['43284 4077 9.0 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_19717742-19718033,1,2408,['43287 4077 9.0 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_19720157-19720448,1,2406,['43290 4062 9.2 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
+_AluY_19722570-19722861,1,2415,['43293 4077 9.0 0.0 0.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999


In [83]:
NA19705_piece1[NA19705_piece1['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_19814849-19815140,2,859,['43381 1327 21.0 0.0 5.4 chrY.09-23.01.HET1_c...,#073B4C,<999
C_AluY_19817265-19817557,2,2410,['43383 418 24.5 3.5 3.8 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19819680-19819971,2,2407,['43386 418 24.0 4.0 3.6 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19822098-19822389,2,2364,['43389 394 25.4 2.9 4.5 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19824455-19824746,2,2401,['43392 410 24.2 3.5 4.1 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19826878-19827167,2,2414,['43395 405 24.7 4.0 3.5 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19829296-19829587,2,2413,['43398 356 20.2 9.0 3.0 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19831710-19832001,2,2407,['43401 420 24.5 3.5 3.6 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999
C_AluY_19834125-19834416,2,2408,['43404 408 24.1 4.1 3.7 chrY.09-23.01.HET1_ce...,#06D6A0,2000-2999


In [84]:
NA19705_piece1.loc['+_AluY_19727412-19727703']['Group']

"['43301 3883 10.7 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19726845 19727412 18323350 False HSATI Satellite 1 568 0 38660 False', '43302 1455 18.5 0.0 5.4 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19727412 19727703 18323059 False AluY SINE/Alu 13 289 12 38661 True', '43303 383 25.8 3.1 4.4 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19727706 19727741 18323021 False (ATATATA)n Simple_repeat 1 1533 0 38662 False', '43304 388 18.5 9.1 3.0 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19727742 19729257 18321505 False (TA)n Simple_repeat 1 1606 0 38663 False']"

In [87]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19705.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705:19727412-19729257"))


>chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705:19727412-19729257
TGGCTCAGGCCTGTAATCCCGGCAAATTTGGAGGCCAAGGCAAGGGGATCACAAGGTGAA
GAGATCAAGACCATCCTGGCCAATACAGTGAAACGCTGTCATTACTAAAAATCCAAAAAA
TTAGCTAGGCATGGTGGCAGCTGGTGTAGTCCCAGTGTGGTGTAGTCCCAGCTACCTGGG
AGGCTGAGGGAGGACAATCGCTTGGACCCGTGAGGCAGATGTTGCAGTGAGCAAAGATGG
CGCCATTGTACTCCACCCTGTGCAACAGGACAAGACTCTGTCTAAAAAAAAATTATAGAT
GATATATATTACATGTTATGTGCTATGCCTTATATGTAACATATAACCTCATATATTTTA
TATGTCATAGTATATAATATTTATCCTGTGTCAGCTTAAATATACTATATATATTACATA
ACATATATAATACATGATACATATTATACATCTTATATATGAAATTATATAATTATACGT
AACATAATACACATTTATATGTATTATGATATATAACAAAGTTACTATATCATATATGAT
ATATAATATATATATTTGTAGTACATATTATAATTTTTATATATTATAATTTATATAATG
ATATAATATTTTATAGAATATAAATAATTATATATGATTATATAATTCTATATATTATAA
ATGAAAATATGTAGAATTACATGTTTATCTAGTTATATGTATAATAATATAATTATATAT
ATAGTTATATATATCACATAATAGACAACATACATATTATATAATGTATAATATATATTA
TAGAATATATGATACATGATATATATGATATTTAATATAACATAATATAATCTATATTAT
AATATTATGTATGTTACTTATATTGGGTGATATGTAATATATATTATGTAATATGAAATA
AT

In [85]:
NA19705_piece1.loc['C_AluY_19814849-19815140']['Group']

"['43381 1327 21.0 0.0 5.4 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19814849 19815140 18235622 True AluY SINE/Alu 12 289 13 38737 True', '43382 3940 10.4 0.0 0.0 chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705 19815140 19815707 18235055 True HSATI Satellite 0 568 1 38738 False']"

In [88]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19705.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705:19813849-19815140"))


>chrY.09-23.01.HET1_centro-HET3_Yq.FW.unassigned-0001118.NA19705:19813849-19815140
AATGGAATGGAATAGAAAGGAATGCAATGAAATGGAATGGAAAGGATTCGAATGGAATGC
AATCGAATGGAATGGAATCGAACGGAATGGAATAAAATGGAAGAAAACTGGCAAGAAATG
GAATCGAAATGAATGGAGTGTTATGGAACGGACTCAAAAGGAATTGAATGTAATAGAATG
GAGTGGAGTGGACTCGAATATAATGGACTGGAATGGAATGAAATCACATGGATTGGGAAC
GAATGGAATGGAATGGAAAGGAATGGAATCGAATGGAAAGGAATCGAATGGAAGGGAATG
AAATTGAATGGACTATATATTTTGTTATATATACTTTTATAACATAATTTACATAAAATA
ACATAATGTAATATATATTATATTATTTCATATTACATAATATATATTACATATCACCCA
ATATAAGTAACATACATAATATTATAATATAGATTATATTATGTTATATTAAATATCATA
TATATCATGTATCATATATTCTATAATATATATTATATATTATATAATATGTATGTTGTC
TATTATGTGATATATATAACTATATATATAATTATATTATTATACATATTATACATATAA
CTAGATAAACATGTAATTCTACATATTTTCGTTTATAATATATAGAATTATATAATCATA
TATAATTATTTATATTCTATAAAAAATTATATCATTGTATAAATTATAATATATAAAAAT
TATAATATGTACTACAAATATATATATTATATATCATATATGATATAGTACCTTTGTTAT
ATATCATAATACATATAAATGTGTTTTATGTTATCTATAATTATCTAATTTCTTATATAA
GGTGTATAATATGTATCATATATTATATATGTTATGTAATATATATAGTATATATAAGAT
GA

In [89]:
############################## 3rd Breakpoint ##################################

In [90]:
NA19705_piece2[NA19705_piece2['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_2931113-2931404,2,1609,['61043 185 24.6 4.0 4.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_2933535-2933825,2,2414,['61046 375 25.7 4.0 3.4 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2935956-2936247,2,2415,['61051 359 26.4 3.1 4.3 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2938373-2938664,2,2410,['61056 428 23.9 3.8 3.6 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2940770-2941061,2,2390,['61059 404 24.1 3.7 4.1 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2943183-2943474,2,2406,['61062 417 25.0 3.3 3.6 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2945743-2946034,2,2553,['61065 421 18.8 9.2 3.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2948163-2948454,2,2413,['61068 394 18.7 9.1 2.9 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2950581-2950872,2,2411,['61071 392 18.8 9.1 2.9 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [96]:
NA19705_piece2.loc['C_AluY_2967505-2967796']['Group']

"['61094 376 19.4 8.9 3.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2965951 2967503 1648589 False (TA)n Simple_repeat 1 1642 0 2716 False', '61095 1441 18.8 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2967505 2967796 1648296 True AluY SINE/Alu 12 289 13 2717 True', '61096 4014 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2967796 2968363 1647729 True HSATI Satellite 0 568 1 2718 False']"

In [92]:
NA19705_piece2.loc['C_AluY_2969928-2970217']['Group']

"['61097 398 25.4 3.4 3.9 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2968371 2969925 1646167 False (TATATAA)n Simple_repeat 1 1548 0 2719 False', '61098 1432 18.6 0.0 5.5 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2969928 2970217 1645875 True AluY SINE/Alu 14 287 13 2720 True', '61099 4043 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2970217 2970784 1645308 True HSATI Satellite 0 568 1 2721 False']"

In [95]:
2970785-2970784

1

In [93]:
NA19705_piece2.loc['+_AluY_2971455-2971733']['Group']

"['61100 156 19.4 9.6 2.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2970785 2971451 1644641 False (AT)n Simple_repeat 1 711 0 2722 False', '61101 1345 19.4 0.0 5.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2971455 2971733 1644359 False AluY SINE/Alu 26 289 12 2723 False', '61102 387 19.2 9.1 2.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2971735 2973284 1642808 False (TA)n Simple_repeat 1 1646 0 2724 False']"

In [97]:
NA19705_piece2.loc['+_AluY_2973859-2974150']['Group']

"['61103 3956 10.0 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2973292 2973859 1642233 False HSATI Satellite 1 568 0 2725 False', '61104 1390 19.2 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2973859 2974150 1641942 False AluY SINE/Alu 13 289 12 2726 True', '61105 401 18.8 9.1 2.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 2974152 2975709 1640383 False (TA)n Simple_repeat 1 1653 0 2727 False']"

In [99]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19705.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705:2970785-2973284"))


In [100]:
NA19705_piece2[NA19705_piece2['RepeatBlock']==3]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_3475071-3475362,3,2380,['61481 3930 10.4 0.0 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3477458-3477749,3,3695,['61484 4016 9.3 0.0 0.2 chrY.23-24.01.HET3_Yq...,#FFD166,3000-3999
+_AluY_3481159-3481450,3,2407,['61487 4005 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3483572-3483863,3,2410,['61490 3839 10.8 0.2 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3485990-3486281,3,2413,['61493 4016 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3488410-3488701,3,2413,['61496 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3490829-3491120,3,1749,['61501 4035 9.2 0.2 0.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_3494149-3494440,3,2409,['61505 402 25.1 3.5 3.8 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_3496569-3496860,3,2413,['61508 410 24.9 3.4 3.9 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [101]:
NA19705_piece2.loc['+_AluY_3490829-3491120']['Group']

"['61501 4035 9.2 0.2 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3490263 3490829 1125263 False HSATI Satellite 1 568 0 3114 False', '61502 1409 18.8 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3490829 3491120 1124972 False AluY SINE/Alu 13 289 12 3115 True', '61503 222 19.2 8.1 3.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3491122 3492011 1124081 False (TA)n Simple_repeat 1 928 0 3116 False']"

In [103]:
3492599-3492011

588

In [102]:
NA19705_piece2.loc['C_AluY_3494149-3494440']['Group']

"['61505 402 25.1 3.5 3.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3492599 3494148 1121944 False (TATATAA)n Simple_repeat 1 1547 0 3118 False', '61506 1439 18.5 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3494149 3494440 1121652 True AluY SINE/Alu 12 289 13 3119 True', '61507 3985 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705 3494440 3495007 1121085 True HSATI Satellite 0 568 1 3120 False']"

In [105]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19705.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000115.NA19705:3490829-3494440"))


In [94]:
################################### 4th Breakpoint ################################

In [91]:
NA19705_piece2[NA19705_piece2['RepeatBlock']==3]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_3475071-3475362,3,2380,['61481 3930 10.4 0.0 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3477458-3477749,3,3695,['61484 4016 9.3 0.0 0.2 chrY.23-24.01.HET3_Yq...,#FFD166,3000-3999
+_AluY_3481159-3481450,3,2407,['61487 4005 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3483572-3483863,3,2410,['61490 3839 10.8 0.2 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3485990-3486281,3,2413,['61493 4016 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3488410-3488701,3,2413,['61496 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3490829-3491120,3,1749,['61501 4035 9.2 0.2 0.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_3494149-3494440,3,2409,['61505 402 25.1 3.5 3.8 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_3496569-3496860,3,2413,['61508 410 24.9 3.4 3.9 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [ ]:
################################################################################################################

In [ ]:
#Piece 2
#RB 2
#RB 3

In [46]:
NA19347_piece2[NA19347_piece2['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_2887415-2887706,2,1609,['67139 185 24.6 4.0 4.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_2889837-2890127,2,2414,['67142 378 25.6 4.0 3.4 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2892258-2892549,2,2415,['67147 359 26.4 3.1 4.3 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2894677-2894968,2,2412,['67152 397 24.2 4.2 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2897097-2897388,2,2413,['67157 428 23.8 3.7 3.7 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2899513-2899804,2,2409,['67160 430 23.9 3.8 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2901910-2902201,2,2390,['67163 404 24.1 3.7 4.1 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2904323-2904614,2,2406,['67166 417 25.0 3.3 3.6 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2906883-2907174,2,2553,['67169 421 18.8 9.2 3.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [47]:
NA19347_piece2.loc['C_AluY_2931068-2931357']['Group']

"['67201 395 25.5 3.4 3.9 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2929511 2931065 1559492 False (TATATAA)n Simple_repeat 1 1548 0 44094 False', '67202 1432 18.6 0.0 5.5 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2931068 2931357 1559200 True AluY SINE/Alu 14 287 13 44095 True', '67203 4043 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2931357 2931924 1558633 True HSATI Satellite 0 568 1 44096 False']"

In [48]:
NA19347_piece2.loc['+_AluY_2932595-2932873']['Group']

"['67204 154 19.6 9.6 2.8 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2931925 2932591 1557966 False (AT)n Simple_repeat 1 711 0 44097 False', '67205 1320 19.8 0.0 5.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2932595 2932873 1557684 False AluY SINE/Alu 26 289 12 44098 False', '67206 387 19.2 9.1 2.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2932875 2934424 1556133 False (TA)n Simple_repeat 1 1646 0 44099 False']"

In [49]:
NA19347_piece2.loc['+_AluY_2934999-2935290']['Group']

"['67207 3956 10.0 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2934432 2934999 1555558 False HSATI Satellite 1 568 0 44100 False', '67208 1390 19.2 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2934999 2935290 1555267 False AluY SINE/Alu 13 289 12 44101 True', '67209 403 18.7 9.1 2.8 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 2935292 2936849 1553708 False (TA)n Simple_repeat 1 1653 0 44102 False']"

In [52]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19347.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347:2931925-2934424"))


>chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347:2931925-2934424
ATATACGTATACATTATGTATATAATAGTATACAGTATCATATATATAATATATATAACC
TATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATAAT
AATTGATAATATATAAGTTTTATATTTATACAATATGACATATAGTATATATTATAATTT
ATGACATATATTATGTATGTTATATATCATATATAGAATATATTATACGGTTAATATACG
TATTAAATATGAATTATACACTATATATATTATATATAATATATCTGTTATATATAATAC
TCATTATGCATAATATATATATTATGTCTATGTTATATATAGTATAAAACAACATATAAT
AATATATAATTGCATATATATAATGCGACATATAATATATATTATATTTTATGACATATG
GAGTATATATTATATATTATGTATAATACGTTGTATGTATAATACCTATATTAAATATAT
ATTATATGACTATGTTATTAATTACATGTCACATATGTTATATATTATATATTTTACATA
GAATATACCGGTTACATACAATATATACTATGTTACCTATAATGTATAATTTATTACATG
TAAAATAAAGAATGCAATATATTATGTATAATTTATAATATACAATATGTGGAATGTGAT
ATATATAACATAATCCCAGCAAATTTGGAGGCCAAGGCAAGGGGATCACAAGGTGAAGAG
ATCAAGACCATCCTGGCCAATACAGTGACACGCTGTCACTACTAAAAATCCAAAAAATTA
GCTAGGCATGGTGGCAGCTGGTGTAGTCCCAGTGTGGTGTAGTCCCAGCTACCTGGGAGG
CTGAGGGAGGACAATCGCTTGGACCCATGAGGCAGATGTTGCAGTGAGCAAAGATGGTGC
CATTGTACCCC

In [173]:
#################################### 4th Breakpoint ##############################

In [174]:
NA19347_piece2[NA19347_piece2['RepeatBlock']==3]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_3386113-3386404,3,2380,['67351 3930 10.4 0.0 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3388500-3388791,3,3695,['67354 4016 9.3 0.0 0.2 chrY.23-24.01.HET3_Yq...,#FFD166,3000-3999
+_AluY_3392201-3392492,3,2407,['67357 4005 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3394614-3394905,3,2410,['67360 3839 10.8 0.2 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3397032-3397323,3,2413,['67363 4016 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3399452-3399743,3,2413,['67366 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3401872-3402163,3,1750,['67371 4047 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_3405193-3405484,3,2410,['67375 399 25.1 3.5 3.8 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_3407609-3407900,3,2409,['67378 414 24.6 3.8 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [176]:
NA19347_piece2.loc['+_AluY_3399452-3399743']['Group']

"['67366 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3398885 3399452 1091105 False HSATI Satellite 1 568 0 44254 False', '67367 1445 18.1 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3399452 3399743 1090814 False AluY SINE/Alu 13 289 12 44255 True', '67368 367 19.3 9.3 2.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3399745 3400596 1089961 False (TA)n Simple_repeat 1 897 0 44256 False', '67369 27 8.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3400597 3400635 1089922 False (ATATATC)n Simple_repeat 1 39 0 44257 False', '67370 367 19.3 9.3 2.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3400636 3401297 1089260 False (TA)n Simple_repeat 2 715 0 44258 False']"

In [177]:
NA19347_piece2.loc['+_AluY_3401872-3402163']['Group']

"['67371 4047 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3401305 3401872 1088685 False HSATI Satellite 1 568 0 44259 False', '67372 1409 18.8 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3401872 3402163 1088394 False AluY SINE/Alu 13 289 12 44260 True', '67373 222 19.2 8.1 3.7 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3402165 3403054 1087503 False (TA)n Simple_repeat 1 928 0 44261 False']"

In [175]:
NA19347_piece2.loc['C_AluY_3405193-3405484']['Group']

"['67375 399 25.1 3.5 3.8 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3403642 3405192 1085365 False (TATATAA)n Simple_repeat 1 1547 0 44263 False', '67376 1435 18.5 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3405193 3405484 1085073 True AluY SINE/Alu 12 289 13 44264 True', '67377 4008 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347 3405484 3406051 1084506 True HSATI Satellite 0 568 1 44265 False']"

In [179]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19347.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.FW.unassigned-0000876.NA19347:3401872-3405484"))


In [ ]:
####################################################################################

In [161]:
NA19317_piece2[NA19317_piece2['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
C_AluY_2823473-2823764,2,1609,['65579 185 24.6 4.0 4.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_2825895-2826185,2,2414,['65582 378 25.6 4.0 3.4 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2828316-2828607,2,2415,['65587 359 26.4 3.1 4.3 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2830735-2831026,2,2412,['65592 397 24.2 4.2 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2833155-2833446,2,2413,['65597 428 23.8 3.7 3.7 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2835571-2835862,2,2409,['65600 430 23.9 3.8 3.5 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2837968-2838259,2,2390,['65603 404 24.1 3.7 4.1 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2840381-2840672,2,2406,['65606 417 25.0 3.3 3.6 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_2842941-2843232,2,2553,['65609 421 18.8 9.2 3.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [162]:
NA19317_piece2.loc['C_AluY_2867126-2867415']['Group']

"['65641 395 25.5 3.4 3.9 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2865569 2867123 1559401 False (TATATAA)n Simple_repeat 1 1548 0 2684 False', '65642 1432 18.6 0.0 5.5 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2867126 2867415 1559109 True AluY SINE/Alu 14 287 13 2685 True', '65643 4043 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2867415 2867982 1558542 True HSATI Satellite 0 568 1 2686 False']"

In [163]:
NA19317_piece2.loc['+_AluY_2868653-2868931']['Group']

"['65644 154 19.6 9.6 2.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2867983 2868649 1557875 False (AT)n Simple_repeat 1 711 0 2687 False', '65645 1320 19.8 0.0 5.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2868653 2868931 1557593 False AluY SINE/Alu 26 289 12 2688 False', '65646 387 19.2 9.1 2.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2868933 2870482 1556042 False (TA)n Simple_repeat 1 1646 0 2689 False']"

In [164]:
NA19317_piece2.loc['+_AluY_2871057-2871348']['Group']

"['65647 3956 10.0 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2870490 2871057 1555467 False HSATI Satellite 1 568 0 2690 False', '65648 1390 19.2 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2871057 2871348 1555176 False AluY SINE/Alu 13 289 12 2691 True', '65649 403 18.7 9.1 2.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 2871350 2872907 1553617 False (TA)n Simple_repeat 1 1653 0 2692 False']"

In [165]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19317.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317:2867982-2870490"))


>chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317:2867982-2870490
AATATACGTATACATTATGTATATAATAGTATACAGTATCATATATATAATATATATAAC
CTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATAA
TAATTGATAATATATAAGTTTTATATTTATACAATATGACATATAGTATATATTATAATT
TATGACATATATTATGTATGTTATATATCATATATAGAATATATTATACGGTTAATATAC
GTATTAAATATGAATTATACACTATATATATTATATATAATATATCTGTTATATATAATA
CTCATTATGCATAATATATATATTATGTCTATGTTATATATAGTATAAAACAACATATAA
TAATATATAATTGCATATATATAATGCGACATATAATATATATTATATTTTATGACATAT
GGAGTATATATTATATATTATGTATAATACGTTGTATGTATAATACCTATATTAAATATA
TATTATATGACTATGTTATTAATTACATGTCACATATGTTATATATTATATATTTTACAT
AGAATATACCGGTTACATACAATATATACTATGTTACCTATAATGTATAATTTATTACAT
GTAAAATAAAGAATGCAATATATTATGTATAATTTATAATATACAATATGTGGAATGTGA
TATATATAACATAATCCCAGCAAATTTGGAGGCCAAGGCAAGGGGATCACAAGGTGAAGA
GATCAAGACCATCCTGGCCAATACAGTGACACGCTGTCACTACTAAAAATCCAAAAAATT
AGCTAGGCATGGTGGCAGCTGGTGTAGTCCCAGTGTGGTGTAGTCCCAGCTACCTGGGAG
GCTGAGGGAGGACAATCGCTTGGACCCATGAGGCAGATGTTGCAGTGAGCAAAGATGGTG
CCATTGTACCC

In [166]:
#################################### 4th Breakpoint ##################################

In [167]:
NA19317_piece2[NA19317_piece2['RepeatBlock']==3]

,RepeatBlock,Lengths,Group,Color,Block_Size
Alu,,,,,
+_AluY_3322171-3322462,3,2380,['65901 3930 10.4 0.0 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3324558-3324849,3,3695,['65904 4016 9.3 0.0 0.2 chrY.23-24.01.HET3_Yq...,#FFD166,3000-3999
+_AluY_3328259-3328550,3,2407,['65907 4005 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3330672-3330963,3,2410,['65910 3839 10.8 0.2 0.0 chrY.23-24.01.HET3_Y...,#06D6A0,2000-2999
+_AluY_3333090-3333381,3,2413,['65913 4016 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3335510-3335801,3,2413,['65916 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
+_AluY_3337930-3338221,3,1750,['65921 4047 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq...,#118AB2,1000-1999
C_AluY_3341250-3341541,3,2409,['65925 404 25.0 3.5 3.8 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999
C_AluY_3343670-3343961,3,2413,['65928 410 24.9 3.4 3.9 chrY.23-24.01.HET3_Yq...,#06D6A0,2000-2999


In [168]:
NA19317_piece2.loc['+_AluY_3335510-3335801']['Group']

"['65916 4020 9.5 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3334943 3335510 1091014 False HSATI Satellite 1 568 0 2952 False', '65917 1445 18.1 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3335510 3335801 1090723 False AluY SINE/Alu 13 289 12 2953 True', '65918 367 19.3 9.3 2.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3335803 3336654 1089870 False (TA)n Simple_repeat 1 897 0 2954 False', '65919 27 8.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3336655 3336693 1089831 False (ATATATC)n Simple_repeat 1 39 0 2955 False', '65920 367 19.3 9.3 2.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3336694 3337355 1089169 False (TA)n Simple_repeat 2 715 0 2956 False']"

In [169]:
NA19317_piece2.loc['+_AluY_3337930-3338221']['Group']

"['65921 4047 9.3 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3337363 3337930 1088594 False HSATI Satellite 1 568 0 2957 False', '65922 1409 18.8 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3337930 3338221 1088303 False AluY SINE/Alu 13 289 12 2958 True', '65923 222 19.2 8.1 3.7 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3338223 3339112 1087412 False (TA)n Simple_repeat 1 928 0 2959 False']"

In [170]:
NA19317_piece2.loc['C_AluY_3341250-3341541']['Group']

"['65925 404 25.0 3.5 3.8 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3339700 3341249 1085275 False (TATATAA)n Simple_repeat 1 1547 0 2961 False', '65926 1439 18.5 0.0 5.4 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3341250 3341541 1084983 True AluY SINE/Alu 12 289 13 2962 True', '65927 3985 9.7 0.0 0.0 chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317 3341541 3342108 1084416 True HSATI Satellite 0 568 1 2963 False']"

In [172]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/NA19317.HIFIRW.ONTUL.na.chrY.fasta', "chrY.23-24.01.HET3_Yq-PAR2.RV.unassigned-0000495.NA19317:3337930-3341541"))


# Breakpoints

In [3]:
HC02666_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HC02666/HC02666_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
HG01890_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01890/HG01890_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
#HG00358_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG00358/HG00358_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
HG02011_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG02011/HG02011_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
HG01106_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01106/HG01106_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
T2T_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/T2T/T2T_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])
#HG01952_df = pd.read_csv("/home/mark/Desktop/chromosome_y/Dataframes/HG01952/HG01952_rbDF2_Kmode.csv").set_index("Alu")#.drop(columns=['Unnamed: 0'])

In [11]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(int(HC02666.iloc[1]['Start'])-500)+"-"+str(HC02666.iloc[1]['Start']))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30356492-30356992


In [12]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(HC02666.iloc[1]['End'])+"-"+str(HC02666.iloc[1]['End']+500))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30390716-30391216


In [ ]:
################################################

In [45]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(int(HC02666.iloc[39]['Start'])-500)+"-"+str(HC02666.iloc[39]['Start']))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52749841-52750341


In [46]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(HC02666.iloc[39]['End'])+"-"+str(HC02666.iloc[39]['End']+500))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52911110-52911610


In [ ]:
################################################

In [47]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(int(HC02666.iloc[41]['Start'])-500)+"-"+str(HC02666.iloc[41]['Start']))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53440452-53440952


In [48]:
print("chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:"+str(HC02666.iloc[41]['End'])+"-"+str(HC02666.iloc[41]['End']+500))

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53489176-53489676


## Compare the Breakpoints found

In [4]:
HG01890_df[HG01890_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_29825231-29825519,0,2216,['45510 373 24.0 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29827646-29827937,0,2411,['45513 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29830064-29830355,0,2411,['45516 405 24.4 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29832473-29832761,0,2399,['45519 363 27.6 2.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29834883-29835171,0,2403,['45522 410 24.2 4.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29837299-29837589,0,2411,['45525 378 19.1 9.1 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29839713-29840000,0,2408,['45529 397 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29842134-29842422,0,2411,['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_29846411-29846702,0,2406,['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22


In [5]:
# HC02666
HC02666_df[HC02666_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_30358347-30358634,0,2214,['45593 372 23.4 4.5 3.3 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30360759-30361047,0,2402,['45596 372 19.8 8.9 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30363169-30363457,0,2403,['45599 400 24.7 3.9 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30365580-30365868,0,2404,['45602 379 26.2 3.0 4.2 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30367996-30368286,0,2418,['45605 390 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30370411-30370698,0,2409,['45608 425 23.8 3.7 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324
C_AluY_30372829-30373091,0,1855,['45611 1214 21.0 0.0 6.0 chrY.01-24.01.PAR1-P...,1000-1999,#F6C324
+_AluY_30375259-30375550,0,2399,['45613 3890 10.5 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#F6C324
+_AluY_30377670-30377961,0,2406,['45616 4003 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324


In [13]:
#with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/HC02666_TheGap.fasta", 'a+') as file:
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29840579-29848249"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30371269-30377095"))
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30374697-30379508"))

#file.close()

In [24]:
with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/HG01890_Comparison_Gap_BothGenomes.fasta", 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29838164-29850656"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30368861-30379508"))
file.close()

In [19]:
HC02666_df.loc['C_AluY_30370411-30370698']['Group']

"['45608 425 23.8 3.7 3.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30368861 30370410 24412047 False (TATATAA)n Simple_repeat 1 1547 0 36449 False', '45609 1345 19.8 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30370411 30370698 24411759 True AluY SINE/Alu 12 289 17 36450 False', '45610 4063 9.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30370702 30371269 24411188 True HSATI Satellite 0 568 1 36451 False']"

In [20]:
HC02666_df.loc['C_AluY_30372829-30373091']['Group']

"['45611 1214 21.0 0.0 6.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30372829 30373091 24409366 True AluY SINE/Alu 12 289 42 36452 False', '45612 422 24.1 3.9 3.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30373133 30374683 24407774 False (ATTATAT)n Simple_repeat 1 1556 0 36453 False']"

In [22]:
HC02666_df.loc['+_AluY_30375259-30375550']['Group']

"['45613 3890 10.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30374697 30375259 24407198 False HSATI Satellite 6 568 0 36454 False', '45614 1403 19.2 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375259 30375550 24406907 False AluY SINE/Alu 13 289 12 36455 True', '45615 388 19.0 8.9 2.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375552 30377095 24405362 False (TA)n Simple_repeat 1 1634 0 36456 False']"

In [23]:
HC02666_df.loc['+_AluY_30377670-30377961']['Group']

"['45616 4003 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377103 30377670 24404787 False HSATI Satellite 1 568 0 36457 False', '45617 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377670 30377961 24404496 False AluY SINE/Alu 13 289 12 36458 True', '45618 389 19.2 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377963 30379508 24402949 False (TA)n Simple_repeat 1 1632 0 36459 False']"

In [10]:
HC02666_df.loc['+_AluY_30375259-30375550']['Group']

"['45613 3890 10.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30374697 30375259 24407198 False HSATI Satellite 6 568 0 36454 False', '45614 1403 19.2 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375259 30375550 24406907 False AluY SINE/Alu 13 289 12 36455 True', '45615 388 19.0 8.9 2.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375552 30377095 24405362 False (TA)n Simple_repeat 1 1634 0 36456 False']"

In [11]:
HC02666_df.loc['+_AluY_30377670-30377961']['Group']

"['45616 4003 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377103 30377670 24404787 False HSATI Satellite 1 568 0 36457 False', '45617 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377670 30377961 24404496 False AluY SINE/Alu 13 289 12 36458 True', '45618 389 19.2 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377963 30379508 24402949 False (TA)n Simple_repeat 1 1632 0 36459 False']"

In [12]:
HG01890_df.loc['+_AluY_29846411-29846702']['Group']

"['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29845844 29846411 16561493 False HSATI Satellite 1 568 0 36538 False', '45537 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846411 29846702 16561202 False AluY SINE/Alu 13 289 12 36539 True', '45538 382 19.4 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846704 29848249 16559655 False (TA)n Simple_repeat 1 1632 0 36540 False']"

In [40]:
with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/HG01890_vs_HC02666_mixed3.fasta", 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842134-29845844"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372829-30374683"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30375259-30377095"))
file.close()

In [33]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30371269-30372829"))


>chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30371269-30372829
AACGTACGTATACATTATGTATATAATAATATACAGTATCATATATATAATATATATAAC
CTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATCA
TAATTGATAATATATAAGTTTTATATTTATACAATATGACATATAGTATATATTATAATT
TATGACATATATGATATATGTCATATATCATATATAGAATATATTATACGGTTAATATTA
GTATTACATATAAATTATACATTATATATATTATATATAATATATCTGTTATATATAATA
CCCATTATGCATAATATATATATTATGACTATGTTATATATAGTATAAAACAACATATAA
TAATATAAAATTGTATATATAAAATGTGACATATAATATATATTATATTTTATGACATAT
CTAATATATATTATATATTATATATAATACGTTGTATGTATAATACCTATATTAAATATA
TATTATATGACTATGTTATTAATTACATGTCACATATGTTATATATTATATATTTTACAT
AGAATATACCGGTTACATATAATATAGACTATGTTACCTATAATGTATAGTTTATTACAT
GTAATATAATGAATGCCATATATTATATATAATTTATAATATACAATATGTGGAATGTGA
TATATATAACATATAGTATATGATATATGATATACAGGATATGATATATAACATACAATA
TATGATACCTAATATATGGTACACAATATATATTTTATGATGTATATTCTATATGTTTTA
TATTCTGTTATATATAACTATATGTAATTATATATAACTGTATAATATTTTATGTAAATA
TGTAATATAATATATATAACTATATAATATATTATATATAACTCTATATTATGTTATATA
TACTTTTATAAC

In [27]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842989-29845844"))


>chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842989-29845844
AACATACGTATACATAATGTATATAATAATATACAGTATCATATATATAATACATATAAC
CTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATAA
TAATTGATAATATATAAGTTTATATTTATACAATATGACATATACCATATATTATATTTT
ATGACATATATAATATATGTTATATATCATATATAGAATATATTATACGGTTAATATACG
TATTAAATATAAATTATACATTATATATATTATATAAAATATATATGTTATATATAATAC
CCATTATACATAATATATATATTATGCCTATGTTATATATAGTATAAAACAACATATAAT
AACATATAATTTAATATATAAAATGTGACATATAATATATATTATATTTTATGACATATG
TAATATAAATTATATATTATACATAATACATTGTATGTATAACACCTATATTAAATATAT
ATTATGTGACTATGTTATTAATTACATGTCACATATGTTATATATTATATATTTTACATA
GAATATAACGGTTACATACAATATATACTATGTTACCCACAATGTATAATTTATTACATG
TAATATAAAGAATGTCATATATTATATATAATTTATAATATATAATATGTGGAATGTGAT
ATATATAACATATAATATATGATATATGATATGCACTATATGAGATATAACATACAATAT
ATGATACCTAATATATATGGTATACAATATATATTTTATGATATGTATTCTATATGTTTT
ATATTCTGTTATATATAACTATTTGAAATTATATATAACTGTATAATATTTTATGTAAAT
ATGTAATATAATATATATAACTATATAATATATTATATATAACTCTATATTATGTTATAT
ATACTTTTATAACATAATTTA

In [17]:
with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/HG01890_vs_HC02666.fasta", 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29835746-29848249"))
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30366436-30379508"))
file.close()

In [ ]:
#HC02666_Start_1stBreakpoint_Coordinate = 30372850
#HG01890_Start_1stBreakpoint_Coordinate = 29844201

In [9]:
#HG01890
HG01890_df[HG01890_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_29825231-29825519,0,2216,['45510 373 24.0 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29827646-29827937,0,2411,['45513 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29830064-29830355,0,2411,['45516 405 24.4 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29832473-29832761,0,2399,['45519 363 27.6 2.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29834883-29835171,0,2403,['45522 410 24.2 4.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29837299-29837589,0,2411,['45525 378 19.1 9.1 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29839713-29840000,0,2408,['45529 397 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29842134-29842422,0,2411,['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_29846411-29846702,0,2406,['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22


In [ ]:
####################################################################################################

In [12]:
30954459+691

30955150

In [ ]:
#HG01890_Start_1stBreakpoint_Coordinate = 29844201 (~ 10 bp before breakpoint)
#HG01890_End_2ndBreakpoint_Coordinate = 29857126 (~ 10 bp after breakpoint)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

#T2T_Start_1stBreakpoint_Coordinate = 28673765
#T2T_End_2ndBreakpoint_Coordinate = 29238515

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#HC02666_Start_1stBreakpoint_Coordinate = 30372850
#HC02666_End_2ndBreakpoint_Coordinate = 30955150

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#


In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29857126
chrY:28673765-29238515    
chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372850-30955150

In [15]:
with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/ThreeGenomes_FullLengthBetween.fasta", 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29857126"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:28673765-29238515"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372850-30955150"))
file.close()

In [16]:
with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/ThreeGenomes_3kLengthBetween.fasta", 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29847201"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:28673765-28676765"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372850-30375850"))
file.close()

In [10]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842201-29844201"))


>chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842201-29844201
ACATCTGCCTCACGGGTCCAAGCGATTGTCCTCCCTCAGCCTCCCAGGTAGCTGGGACTA
CACCACACTGGGACTACACCATCTGCCACCATGCCTAGCTAATTTTTTGGATTTTTAGTA
GTGACAGCCTTACACTGTATTGGCCAGGATGGTCTTGATCTCTTCACCTTGTGATCCCCT
TGCCTTGTCCTCCAAATTTGCTGGGATTAGAGGCCTGAGCCAAGATCCATATTTTTTAAA
TGAAAGAAAATTTCAAGGGTACTCTGCTTGGTACAATAATCAAATGTATAAACTGAGGAA
TAAAACATAACCATGAAACATACTTATAAGTGCATATGGAAAATACCGAGGATAATTTTT
TAAATAACATATTTTGAAAGCCTTAACTAGTAATTTGAAGAGCTGGCATTTGAAAGGCCA
GTATGAATACACCTTCAAAGCAGCAACACAGGATCCCCATGAGAAAAAGCAAACTCAGGG
AAAATAAAACCACAAGGGTTCAATCTCTTCTGACCTTTGAAAGACACAGCACAGACAGTG
GTCCTTAGGACGAAGAGCAGGAGACCCCTAATTCCGTCACCATGGCGATAGGCCACAAAC
ATTGCTGGGTGAGGGTACAATCCACATTGTGAGGTCCAACTGCTGCCAGGAAGACAGGAG
TGCTTTTACATAGACCGAAAGGTCATTGAAGGCTCAGAGTTTTGTTTCAAGAACTGAATC
CCAAGCCCACACCTTATTATGCTGCGCTTCTTAAAACAAGTTATGAGATGGGAAAAAGGA
CACCCACAAACATACGTATACATAATGTATATAATAATATACAGTATCATATATATAATA
CATATAACCTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATA
AAATATAATAATTGATAATAT

In [12]:
32748638-32747363

1275

In [13]:
32750939-32750567

372

In [16]:
2362-459+372+1275

3550

In [22]:
#Rows = 1182-1184
#DYZ1_Yq	1	3583	32743776	32747363
#Yqhet_2k7bp	459	2362	32748638	32750567
#DYZ1_Yq	1	3583	32750939	32754511

print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:32747363-32750939"))


>chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:32747363-32750939
CATTGGAATGGACAGGCCTGGAATAAAGTGGAATGCTACGGTCTCGAATGGAATAAAAAT
GTATGGAATGGAATGCAATGAAACGGAATCGAATGTCATAGAATGTAATGGAATGCAAAA
AAATGGAATCCAAAATCATTGACTGGAAAGGCTGGGTGTCGAAAGGAATTGACTCCAATG
GAATGGAATCGAATGGAATGGAAGTGAATAGAATCGAACTAAATCGAATGGAATGGAATT
GATAGGAACGGAATGGAAAGGAATGCAATGATTTGGCATGGAATGGAATCGAATGGCATC
GAATGGAATGGAATGGAATGCAATGGAATGGAATGTATTAGAATGTAATGAACTTTAATG
GAATGTACTCGAATGGATTCGACTGGAATGGAATGTTCTGGAAGTGAATGGACTCCAATG
GAATGGATTCAAAAGGAATGGAATCGTACGGAATGGAATCTAATGGAATGGAATTAAATG
GAAATGAATCAAATTGAATAGCACGGAATTGAATTGAATGGAATGGAATGCAATGGAATC
TAATGAAACGGAAAGGAAAGGAATGGAATGGAATGGAATGGGCTGGAATGGAAAGGAATC
GAAACGAATGGAATGGAATCGAACTGAAGAGACTGGAATGGAATGCACTGGAATGGAAGG
GAGTGTAATGCAAGGTTCTCGAAAAAAATGGAATCGAATGGAATGGAATTGAATGGAACG
GAATAGAGTCGAATGGAATTGAATGGAATGGAATGGACTAGAGTGAAATGGAATCGAACC
ACAAGGAATGGACAGGAATAGAATGGTCTCGAATTGAATGGAATCGTATGGAATGGCATC
AAACGGAATGGAATGGACAGCCACGGAATGGAATGCACTCGAATGCAATGGAGTCGAAAC
TAATGGACTGGAATAGAATGG

In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29844701
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29856626-29857126

chrY:28673765-28674265 
chrY:29238015-29238515    

chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372850-30373350
chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30954650-30955150

In [ ]:
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29844701"))


In [21]:
#with open("/home/mark/Desktop/chromosome_y/breakpoints/FinalGenomes/ThreeGenomes_OnlyEnds.fasta", 'a+') as file:
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29844201-29844701"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29856626-29857126"))

#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:28673765-28674265"))
    
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:29238015-29238515"))

#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372850-30373350"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HC02666.HIFIRW.ONTUL.na.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30954650-30955150"))

#file.close()


In [ ]:
#HG01890_Start_1stBreakpoint_Coordinate = 29844201 (~ 10 bp before breakpoint)
#HG01890_End_2ndBreakpoint_Coordinate = 29857126 (~ 10 bp after breakpoint)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

#T2T_Start_1stBreakpoint_Coordinate = 28673765
#T2T_End_2ndBreakpoint_Coordinate = 29238515

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#HC02666_Start_1stBreakpoint_Coordinate = 30372850
#HC02666_End_2ndBreakpoint_Coordinate = 30955150

# T2T

In [9]:
T2T_df[T2T_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_28661430-28661721,0,2217,['373 24.5 3.2 3.8 chrY 28660073 28661429 (337...,2000-2999,#FF033E
C_AluY_28663852-28664140,0,2411,['414 24.2 3.9 3.7 chrY 28662297 28663849 (337...,2000-2999,#FF033E
C_AluY_28666267-28666558,0,2411,['407 24.3 3.9 3.8 chrY 28664715 28666266 (337...,2000-2999,#FF033E
C_AluY_28668676-28668963,0,2402,['381 26.8 2.9 3.9 chrY 28667133 28668673 (337...,2000-2999,#FF033E
C_AluY_28671087-28671377,0,2403,['358 20.7 8.6 2.8 chrY 28669542 28671048 (337...,2000-2999,#FF033E
C_AluY_28673501-28673763,0,2430,['1228 20.2 0.0 6.0 chrY 28673501 28673763 (33...,2000-2999,#FF033E
+_AluY_28675930-28676221,0,1839,['1403 19.2 0.0 5.4 chrY 28675930 28676221 (33...,1000-1999,#FF033E
+_AluY_28678343-28678634,0,2400,['4026 9.5 0.0 0.0 chrY 28677776 28678343 (337...,2000-2999,#FF033E
+_AluY_28680761-28681053,0,2408,['4022 9.5 0.0 0.0 chrY 28680194 28680761 (337...,2000-2999,#FF033E


In [12]:
T2T_df.loc['C_AluY_28671087-28671377']['Group']

"['358 20.7 8.6 2.8 chrY 28669542 28671048 (33788981) + (TA)n Simple_repeat 1 1592 (0) 36037', '1397 18.5 0.3 5.4 chrY 28671087 28671377 (33788652) C AluY SINE/Alu (12) 289 13 36038 *', '4088 9.0 0.0 0.0 chrY 28671377 28671944 (33788085) C HSATI Satellite (0) 568 1 36039']"

In [21]:
28673501-28671944

1557

In [10]:
T2T_df.loc['C_AluY_28673501-28673763']['Group']

"['1228 20.2 0.0 6.0 chrY 28673501 28673763 (33786266) C AluY SINE/Alu (12) 289 42 36040', '3887 10.3 0.2 0.0 chrY 28675369 28675930 (33784099) + HSATI Satellite 6 568 (0) 36041']"

In [20]:
28675930-28675930

0

In [13]:
T2T_df.loc['+_AluY_28675930-28676221']['Group']

"['1403 19.2 0.0 5.4 chrY 28675930 28676221 (33783808) + AluY SINE/Alu 13 289 (12) 36042 *', '382 19.3 8.9 3.0 chrY 28676223 28677768 (33782261) + (TA)n Simple_repeat 1 1634 (0) 36043']"

In [14]:
T2T_df.loc['+_AluY_28678343-28678634']['Group']

"['4026 9.5 0.0 0.0 chrY 28677776 28678343 (33781686) + HSATI Satellite 1 568 (0) 36044', '1465 18.1 0.0 5.4 chrY 28678343 28678634 (33781395) + AluY SINE/Alu 13 289 (12) 36045 *', '384 18.6 8.7 3.5 chrY 28678636 28680175 (33779854) + (TA)n Simple_repeat 1 1618 (0) 36046']"

In [25]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:28671944-28675930"))


>chrY:28671944-28675930
AACATACGTATACATTATGTATATAATAATATACAGTATCATATATATAATATATATAAC
CTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATAA
TAATTGATAATATATAAGTTTATATTTATACAAGATGACATATAGTATATATTATAATTT
ATGACATATATAATATATGTTATATATCATATATAGAATATATTATACGGTTAATATACG
TATTAAATATGTTATACATTATATATATTATATATAATATATCTGTTATATATAATACCC
ATTATACATAATATATATATTATGTCTATGTTATATATAGTATAAAACAACATATAATAA
TATATAATTGTATATATAAAATGTGACATATAATATATATTTTATTTTATGACATATGTA
ATATATATTATATATTATATATAATACGTTGTAAGTATAATACCTATATTAAATATATAT
TATATGACTATGTTATTAATTACATGTCACATATGTTATATATTATATACTTTACATAGA
ATATACCGGTTACATACAATATATACTATGTTACCTATAATGTATAATTTATTACATGTA
ATATAAAGAATGTCATATATTATATATAATTTATAATATATAATATGTGGAATGTGATAT
ATATAACATATAATATATGATATATGATATACAGTTTATGATATATAACATACAATATAT
GATACATAATATATGTGGTATACAATATATATTTTATGATATGTATTCTATATGTTTTAT
ATTCTGTTATATATAACTATTTGAAATTATATATAACTGTATAATATTTTATGTAAATAT
GTAATATAATATATAACTATATAATATATTATATATAACTATATATTATGTTATATATAC
TTTTATAACATAATTTACATAAAATATCATAATATAATATATATTATATTATTTCATATT


In [24]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/T2T.chrY.fasta', "chrY:28675369-28677768"))


>chrY:28675369-28677768
GGTGCCATTTTTCCATCTCATAACTTGTTTTAAGAAGCGCAGCATAATAACGTGTGGGCT
TGGGATTCAGTTCTTGAAACAAAACTCTGAGCCTTCAATGAACTTTCGGTCTATGTAAAA
GCACTCCTGTCTTCCTGGCAGCAGTTGGACCTCACAATGTGGATTGTGCCCTCACCCAGC
AATGTTTATGCCCTATCGCCATGGTGACGGAATTAAGGGTCTCCTGCTCTTCGTCCTAAG
GACCACTGTCCGTGCTGTGTCTTTCAAAGGTCAGAAGAGATTGAACCTTTGTGGTTTTAT
TTTACCTGAGTTTGCTTTTTCTCATGGGGAACCTGTGTTGCTGCTTTGAAGGTATATTCA
TAATGGCCTTTCAAATGCCAGCTCTTCAAATTACTAGTTAAGGCTTTCAAAATATGTTAT
TTAAAAAATTATCCTCTGTATTTTCCATATGCAGTTATAAGTATGTTTCATGGTTATGTT
TTATTCCTCAGTTTATACATTTGATAATTATACCAAGCAGAGTACCTTTGAAATTTTTTT
TCATTTAAAAATTATGGATCTTGCCTCAGGCCTGTAATCCCAGCAAATTTGGAGGACAAG
GCAAGGGGATCACAAGATGAAGAGATCAAGACCATCCTGGCCAATACAGTGAAAGGCTGT
CACTACTAAAAATCCAAAAAATTAGCTGGGCATGGTGGCAGCTGGTGTAGTCCCAGTGTG
GTGTAGTCCCAGCTACCTGGGAGGCTGAGGGAGGACAATCGTTTGGACCCGTGAGGCAGA
TGTTGCAGTGAGCAAAGATGGCGCCATTGTACTCCACCCTGTGCAACAGGACAAGACTCT
GTCTAAAAAAAAATTATATATGATATATATTACATGATATGTGTTATGCCTTATATGTAA
CATATAACCTCATATATTTTATATGTCATAGTATAAAATATTTATCCTGTGTCATCTTAT


# HG00358

In [75]:
HG00358_df[HG00358_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_27648628-27648919,0,2217,['42438 373 24.1 3.6 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27651050-27651338,0,2411,['42441 411 24.3 3.9 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27653465-27653756,0,2411,['42444 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27655874-27656161,0,2402,['42447 379 26.9 2.9 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27658288-27658576,0,2404,['42450 358 20.7 8.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27660704-27660994,0,2418,['42453 389 19.2 9.0 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27663119-27663406,0,2409,['42456 423 23.9 3.7 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_27665535-27665797,0,3404,['42459 296 27.6 5.4 3.1 chrY.01-24.01.PAR1-PA...,3000-3999,#FF033E
+_AluY_27667964-27668255,0,2398,['42462 3887 10.3 0.2 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#FF033E


In [78]:
HG00358_df.loc['C_AluY_27663119-27663406']['Group']

"['42456 423 23.9 3.7 3.9 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27661569 27663118 31240323 False (TATATAA)n Simple_repeat 1 1547 0 34197 False', '42457 1345 19.8 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27663119 27663406 31240035 True AluY SINE/Alu 12 289 17 34198 False', '42458 4051 9.3 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27663410 27663977 31239464 True HSATI Satellite 0 568 1 34199 False']"

In [79]:
27663986-27663977

9

In [76]:
HG00358_df.loc['C_AluY_27665535-27665797']['Group']

"['42459 296 27.6 5.4 3.1 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27663986 27665532 31237909 False (ATATATC)n Simple_repeat 1 1581 0 34200 False', '42460 1226 20.6 0.0 6.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27665535 27665797 31237644 True AluY SINE/Alu 12 289 42 34201 False', '42461 422 24.1 3.9 3.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27665839 27667389 31236052 False (ATTATAT)n Simple_repeat 1 1556 0 34202 False']"

In [80]:
27667403-27667389

14

In [77]:
HG00358_df.loc['+_AluY_27667964-27668255']['Group']

"['42462 3887 10.3 0.2 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27667403 27667964 31235477 False HSATI Satellite 6 568 0 34203 False', '42463 1403 19.2 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27667964 27668255 31235186 False AluY SINE/Alu 13 289 12 34204 True', '42464 391 18.9 8.9 2.9 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 27668257 27669800 31233641 False (TA)n Simple_repeat 1 1634 0 34205 False']"

In [4]:
import pysam

In [7]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG00358.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358:27663986-27667389"))


In [ ]:
############################################ 2nd breakpoint ######################################################

In [8]:
HG00358_df[HG00358_df['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_28134913-28135204,1,2413,['42731 4013 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_28137333-28137624,1,2411,['42734 4014 9.5 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_28139751-28140042,1,2412,['42737 4065 9.2 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_28142170-28142461,1,2413,['42740 3920 10.2 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#228B22
+_AluY_28144590-28144880,1,2412,['42743 3983 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_28147009-28147300,1,2413,['42746 4012 9.5 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_28149429-28149720,1,2424,['42749 3932 10.0 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#228B22
+_AluY_28151860-28152151,1,2411,['42752 3949 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_28154278-28154569,1,2411,['42755 3983 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E


In [9]:
HG00358_df[HG00358_df['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_28269797-28270088,2,1546,['42831 172 24.4 4.1 3.9 chrY.01-24.01.PAR1-PA...,1000-1999,#FF033E
C_AluY_28272213-28272505,2,2410,['42834 417 24.3 3.6 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28274628-28274919,2,2407,['42837 414 24.0 3.8 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28277046-28277337,2,2364,['42840 392 25.5 2.9 4.5 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28279403-28279694,2,2401,['42843 408 24.3 3.5 4.1 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28281826-28282115,2,2414,['42846 405 24.7 4.0 3.5 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28284244-28284535,2,2413,['42849 355 20.2 8.9 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_28286662-28286953,2,2411,['42852 432 24.6 3.3 3.5 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_28289082-28289373,2,2413,['42855 422 24.2 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22


In [10]:
HG00358_df.loc['+_AluY_28192970-28193261']['Group']

"['42803 3991 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28192403 28192970 30710471 False HSATI Satellite 1 568 0 34523 False', '42804 1462 18.1 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28192970 28193261 30710180 False AluY SINE/Alu 13 289 12 34524 True', '42805 383 25.8 3.1 4.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28193264 28193299 30710142 False (ATATATA)n Simple_repeat 1 1533 0 34525 False']"

In [11]:
HG00358_df.loc['C_AluY_28269797-28270088']['Group']

"['42831 172 24.4 4.1 3.9 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28269110 28269794 30633647 False (TATATAT)n Simple_repeat 1 686 0 34549 False', '42832 1330 20.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28269797 28270088 30633353 True AluY SINE/Alu 12 289 13 34550 True', '42833 3940 10.4 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 28270088 28270655 30632786 True HSATI Satellite 0 568 1 34551 False']"

In [16]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG00358.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358:28192970-28194299"))

In [17]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG00358.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358:28267110-28270088"))

In [121]:
######################### 3rd Breakpoint ###############################

In [122]:
HG00358_df[HG00358_df['RepeatBlock']==35]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_57241620-57241911,35,1608,['66577 188 24.4 4.0 4.0 chrY.01-24.01.PAR1-PA...,1000-1999,#FF033E
C_AluY_57244041-57244331,35,2414,['66580 375 25.7 4.0 3.4 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57246462-57246753,35,2415,['66585 359 26.4 3.1 4.3 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57248881-57249172,35,2412,['66590 402 24.0 4.2 3.5 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57251301-57251592,35,2413,['66595 428 23.8 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_57253717-57254008,35,2409,['66598 428 24.1 3.7 3.5 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57256141-57256429,35,2414,['66601 422 23.7 4.0 3.6 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57258554-57258845,35,2409,['66604 430 23.9 3.8 3.5 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
C_AluY_57260978-57261266,35,2414,['66607 422 23.8 3.8 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E


In [123]:
HG00358_df.loc['C_AluY_57311849-57312138']['Group']

"['66676 420 24.2 4.1 3.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57310295 57311846 1591595 False (TATATAA)n Simple_repeat 1 1562 0 57906 False', '66677 1449 18.2 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57311849 57312138 1591303 True AluY SINE/Alu 14 287 13 57907 True', '66678 3951 9.6 0.5 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57312138 57312702 1590739 True HSATI Satellite 0 568 1 57908 False']"

In [124]:
HG00358_df.loc['+_AluY_57313373-57313651']['Group']

"['66679 157 19.3 9.8 2.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57312703 57313369 1590072 False (AT)n Simple_repeat 1 713 0 57909 False', '66680 1345 19.4 0.0 5.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57313373 57313651 1589790 False AluY SINE/Alu 26 289 12 57910 False', '66681 376 19.6 8.9 2.9 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57313653 57315195 1588246 False (TA)n Simple_repeat 1 1632 0 57911 False']"

In [125]:
HG00358_df.loc['+_AluY_57315770-57316060']['Group']

"['66682 3992 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57315203 57315770 1587671 False HSATI Satellite 1 568 0 57912 False', '66683 1414 18.5 0.3 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57315770 57316060 1587381 False AluY SINE/Alu 13 289 12 57913 True', '66684 387 25.3 3.6 3.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57316099 57317606 1585835 False (ATATATA)n Simple_repeat 1 1505 0 57914 False']"

In [127]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG00358.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358:57312702-57315203"))

In [128]:
############################################## 4th Breakpoint ###########################################

In [129]:
HG00358_df[HG00358_df['RepeatBlock']==36]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_57893149-57893440,36,2380,['67222 3903 10.6 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#FF033E
+_AluY_57895536-57895827,36,3695,['67225 4016 9.3 0.0 0.2 chrY.01-24.01.PAR1-PA...,3000-3999,#FF033E
+_AluY_57899237-57899528,36,2407,['67228 4005 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_57901650-57901941,36,2410,['67231 3854 10.6 0.2 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#FF033E
+_AluY_57904068-57904359,36,2413,['67234 4016 9.5 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_57906488-57906779,36,2413,['67239 3875 10.7 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#FF033E
+_AluY_57908907-57909198,36,2410,['67242 3937 9.9 0.2 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_57911325-57911616,36,2411,['67245 3957 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E
+_AluY_57913743-57914034,36,2413,['67248 4010 9.5 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E


In [131]:
HG00358_df.loc['+_AluY_57918583-57918874']['Group']

"['67256 4008 9.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57918016 57918583 984858 False HSATI Satellite 1 568 0 58459 False', '67257 1446 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57918583 57918874 984567 False AluY SINE/Alu 13 289 12 58460 True', '67258 418 24.6 3.9 3.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57918875 57920430 983011 False (TTATATA)n Simple_repeat 1 1563 0 58461 False']"

In [132]:
HG00358_df.loc['+_AluY_57921006-57921297']['Group']

"['67259 3869 10.6 0.0 0.2 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57920438 57921006 982435 False HSATI Satellite 1 568 0 58462 False', '67260 1431 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57921006 57921297 982144 False AluY SINE/Alu 13 289 12 58463 True', '67261 231 18.6 8.1 3.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57921299 57922188 981253 False (TA)n Simple_repeat 1 928 0 58464 False']"

In [130]:
HG00358_df.loc['C_AluY_57924326-57924617']['Group']

"['67263 406 24.9 3.5 3.8 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57922776 57924325 979116 False (TATATAA)n Simple_repeat 1 1547 0 58466 False', '67264 1408 19.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57924326 57924617 978824 True AluY SINE/Alu 12 289 13 58467 True', '67265 3961 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358 57924617 57925184 978257 True HSATI Satellite 0 568 1 58468 False']"

In [134]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG00358.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-2510.HG00358:57921006-57924617"))

# HG01952

In [22]:
HG01952_df[HG01952_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_1882515-1882806,0,2217,['45240 374 23.9 3.7 3.7 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1884937-1885225,0,2411,['45243 384 18.9 9.1 3.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1887352-1887643,0,2411,['45246 407 24.3 3.9 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1889761-1890048,0,2402,['45249 379 26.9 2.9 3.9 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1892175-1892463,0,2404,['45252 358 20.7 8.6 3.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1894591-1894881,0,2418,['45255 388 18.9 9.1 2.7 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1897006-1897293,0,2409,['45258 393 19.1 8.8 2.9 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_1899422-1899684,0,3404,['45261 296 27.6 5.4 3.1 chrY.21-24.01.AMPL7-P...,3000-3999,#4169e1
+_AluY_1901851-1902142,0,2398,['45264 3887 10.3 0.2 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#4169e1


In [23]:
HG01952_df.loc['C_AluY_1899422-1899684']['Group']

"['45261 296 27.6 5.4 3.1 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1897873 1899419 21581446 False (ATATATC)n Simple_repeat 1 1581 0 34707 False', '45262 1226 20.6 0.0 6.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1899422 1899684 21581181 True AluY SINE/Alu 12 289 42 34708 False', '45263 419 24.2 3.9 3.5 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1899726 1901276 21579589 False (ATTATAT)n Simple_repeat 1 1556 0 34709 False']"

In [24]:
HG01952_df.loc['+_AluY_1901851-1902142']['Group']

"['45264 3887 10.3 0.2 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1901290 1901851 21579014 False HSATI Satellite 6 568 0 34710 False', '45265 1403 19.2 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1901851 1902142 21578723 False AluY SINE/Alu 13 289 12 34711 True', '45266 391 18.9 8.9 2.9 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 1902144 1903687 21577178 False (TA)n Simple_repeat 1 1634 0 34712 False']"

In [30]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01952.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952:1897873-1901276"))

In [ ]:
##################### 2nd breakpoint #############################################3

In [31]:
HG01952_df[HG01952_df['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_2504377-2504668,1,2413,['45616 4038 9.7 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_2506797-2507088,1,2411,['45619 3989 9.7 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_2509215-2509506,1,2412,['45622 4066 9.2 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
+_AluY_2511634-2511925,1,2413,['45625 3968 9.9 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_2514054-2514344,1,2412,['45628 3983 9.7 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_2516473-2516764,1,2413,['45631 4012 9.5 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
+_AluY_2518893-2519184,1,2424,['45634 3961 9.9 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
+_AluY_2521324-2521615,1,2411,['45637 3926 10.0 0.0 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#4169e1
+_AluY_2523742-2524033,1,2411,['45640 3959 10.0 0.0 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#4169e1


In [32]:
HG01952_df[HG01952_df['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_2699850-2700141,2,1546,['45813 172 24.4 4.1 3.9 chrY.21-24.01.AMPL7-P...,1000-1999,#4169e1
C_AluY_2702266-2702558,2,2410,['45816 419 24.2 3.6 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2704681-2704972,2,2407,['45819 416 24.1 4.0 3.6 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2707099-2707390,2,2364,['45822 389 25.5 2.9 4.5 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2709456-2709747,2,2401,['45825 408 24.3 3.5 4.1 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2711879-2712168,2,2414,['45828 405 24.7 4.0 3.5 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2714297-2714588,2,2413,['45831 352 20.4 9.0 3.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2716717-2717009,2,2414,['45834 403 25.0 3.5 3.9 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_2719133-2719424,2,2408,['45837 407 23.9 4.1 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1


In [33]:
HG01952_df.loc['+_AluY_2576958-2577249']['Group']

"['45708 3926 10.4 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2576391 2576958 20903907 False HSATI Satellite 1 568 0 35123 False', '45709 1446 18.8 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2576958 2577249 20903616 False AluY SINE/Alu 13 289 12 35124 True', '45710 379 26.0 3.1 4.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2577252 2577287 20903578 False (ATATATA)n Simple_repeat 1 1533 0 35125 False']"

In [34]:
HG01952_df.loc['C_AluY_2699850-2700141']['Group']

"['45813 172 24.4 4.1 3.9 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2699163 2699847 20781018 False (TATATAT)n Simple_repeat 1 686 0 35223 False', '45814 1330 20.6 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2699850 2700141 20780724 True AluY SINE/Alu 12 289 13 35224 True', '45815 3940 10.4 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 2700141 2700708 20780157 True HSATI Satellite 0 568 1 35225 False']"

In [39]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01952.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952:2576958-2578287"))

In [38]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01952.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952:2698163-2700141"))

In [135]:
################################# 3rd Breakpoint #####################################

In [136]:
HG01952_df[HG01952_df['RepeatBlock']==21]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_22250092-22250383,21,1609,['62842 181 25.0 4.0 4.0 chrY.21-24.01.AMPL7-P...,1000-1999,#4169e1
C_AluY_22252514-22252804,21,2414,['62845 374 25.9 3.9 3.3 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22254935-22255226,21,2415,['62850 381 25.9 3.6 3.5 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22257355-22257646,21,2413,['62855 428 23.8 3.7 3.7 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
C_AluY_22259771-22260062,21,2409,['62858 430 23.9 3.8 3.5 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22262195-22262483,21,2414,['62861 422 24.0 3.7 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22264588-22264879,21,2394,['62864 413 24.3 3.3 4.1 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22267003-22267294,21,2408,['62867 423 24.6 3.2 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
C_AluY_22269563-22269854,21,2553,['62870 421 18.8 9.2 3.0 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324


In [137]:
HG01952_df.loc['C_AluY_22308259-22308548']['Group']

"['62923 395 25.5 3.4 3.9 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22306702 22308256 1172609 False (TATATAA)n Simple_repeat 1 1548 0 52008 False', '62924 1449 18.2 0.0 5.5 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22308259 22308548 1172317 True AluY SINE/Alu 14 287 13 52009 True', '62925 4043 9.3 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22308548 22309115 1171750 True HSATI Satellite 0 568 1 52010 False']"

In [138]:
HG01952_df.loc['+_AluY_22309786-22310064']['Group']

"['62926 157 19.3 9.8 2.7 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22309116 22309782 1171083 False (AT)n Simple_repeat 1 713 0 52011 False', '62927 1353 19.0 0.0 5.7 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22309786 22310064 1170801 False AluY SINE/Alu 26 289 12 52012 False', '62928 378 19.4 8.9 3.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22310066 22311619 1169246 False (TA)n Simple_repeat 1 1644 0 52013 False']"

In [139]:
HG01952_df.loc['+_AluY_22312194-22312485']['Group']

"['62929 4035 9.5 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22311627 22312194 1168671 False HSATI Satellite 1 568 0 52014 False', '62930 1382 19.9 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22312194 22312485 1168380 False AluY SINE/Alu 13 289 12 52015 True', '62931 384 19.0 9.3 2.9 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22312487 22314039 1166826 False (TA)n Simple_repeat 1 1650 0 52016 False']"

In [141]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01952.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952:22309115-22311627"))

In [142]:
####################################### 4th Breakpoint #####################################

In [143]:
HG01952_df[HG01952_df['RepeatBlock']==22]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_22662225-22662516,22,2380,['63177 3930 10.4 0.0 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#4169e1
+_AluY_22664612-22664903,22,3695,['63180 4016 9.3 0.0 0.2 chrY.21-24.01.AMPL7-P...,3000-3999,#4169e1
+_AluY_22668313-22668604,22,2407,['63183 4005 9.7 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_22670726-22671017,22,2410,['63186 3854 10.6 0.2 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#F6C324
+_AluY_22673144-22673435,22,2413,['63189 3957 9.9 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_22675564-22675855,22,2413,['63192 4010 9.5 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_22677984-22678275,22,2415,['63197 4008 9.5 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
+_AluY_22680407-22680698,22,1751,['63200 3884 10.4 0.0 0.2 chrY.21-24.01.AMPL7-...,1000-1999,#F6C324
C_AluY_22683727-22684018,22,2409,['63204 409 24.8 3.5 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1


In [144]:
HG01952_df.loc['+_AluY_22680407-22680698']['Group']

"['63200 3884 10.4 0.0 0.2 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22679839 22680407 800458 False HSATI Satellite 1 568 0 52265 False', '63201 1468 18.8 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22680407 22680698 800167 False AluY SINE/Alu 13 289 12 52266 True', '63202 229 18.7 8.1 3.7 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22680700 22681589 799276 False (TA)n Simple_repeat 1 928 0 52267 False']"

In [145]:
HG01952_df.loc['C_AluY_22683727-22684018']['Group']

"['63204 409 24.8 3.5 3.8 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22682177 22683726 797139 False (TATATAA)n Simple_repeat 1 1547 0 52269 False', '63205 1439 18.5 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22683727 22684018 796847 True AluY SINE/Alu 12 289 13 52270 True', '63206 4010 9.5 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952 22684018 22684585 796280 True HSATI Satellite 0 568 1 52271 False']"

In [148]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01952.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.utig4-9605.HG01952:22679839-22684585"))

# HG02011

In [ ]:
#908	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	1	3130	18532297	18535353
#912	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	1591	3130	18536608	18538109
#916	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_2k7bp	438	2291	18540598	18542440
#919	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	703	3137	18543230	18545627
#923	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	703	3130	18547984	18550374


In [9]:
18540598-18538109

2489

In [20]:
#with open('/home/mark/Desktop/HG02011_Transition.fasta', 'a+') as file:
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18535353-18538109"))
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18538109-18542440"))
    #file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18542440-18545627"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18536608-18538109"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18540598-18542440"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18543230-18545627"))
#    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18540598-18543230"))
#file.close()


In [22]:
#46	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_3k1bp	2	3119	17276196	17279311	17276195	17279312	40624160	+	0	2710	609	3120	99.94	1
#83	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ18_Yq	277	2862	17276791	17279311	17276787	17279313	40624160	+	0	1989.7	482.5	2864	90.29	0
#48	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_3k1bp	3	2204	17279312	17281497	17279310	17281499	40624160	+	0	1786	423.9	3120	70.58	1
#93	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ18_Yq	282	2862	17279911	17282478	17279908	17282480	40624160	+	0	2052.9	475	2864	90.12	0
#13	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_3k1bp	2	2205	17282478	17284662	17282477	17284663	40624160	+	0	1762	420.8	3120	70.64	1


600

In [30]:
981+2201

3182

In [31]:
#2.7 - DYZ1 border - looks like its more a DYZ1 call missing
#1	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_2k7bp	2	1868	17414216	17416077	17414215	17416078	40624160	+	0	1596.1	337.7	2767	67.47	1
#4334	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	868	2405	17414521	17416059	17414515	17416061	40624160	-	0	1080.9	270.7	3584	42.91	0
# HERE IS THE MISSING CALL - 22	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	1	1249	17416045	17417293	17416043	17417293	40624160	-	0	1039.5	219.6	3584	34.85	0
#170	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ1_Yq	1	3583	17417295	17420821	17417294	17420821	40624160	-	0	3018.9	565.6	3584	99.97	1

#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:17416077-17417295"))

>chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:17416077-17417295
AAGGGAATGAAATTGAATCAACAGGAATGGAAGGGAATAGAATAGACGGTAATGGAATGG
ACTCGAATAGAATGGACACGAATGCTCAAATTGAATGGACTCGAATGGAATGGAAACGAA
AGGAATGTCATCGAATGGAATTGAATCGATTGGAATGAAATCGCATAGAATGGAGTGGAA
TCAAATGGAATTGAATCGAAAGGAAAGGAAGTGAATGGACTGAAATGAAATGGAATGGAA
TTGAATGGAAAGTAATGCAATGGAATAGAATGGAACGAAATTTCACGGAATGGAAGCAAA
CTGAATGGAATCAAATCAATGGAATATAATCAAATGGAATGGAAAGGAATTGAATGGAGT
AGAAGGGATTGGATGGGATTGGAATGAAATGTACTGGAAAGGACTCGAATTTCATGAAAC
GGAATGGAATGAATTGGAACGGAATGGACTCGAATGGAATGGAATGTCATGGAATGGCAT
CAAATGGAATGGCATCAAATGGAATGGAATCGAATGCAATGGAATGCTATGGAATGGAAT
GGAATGCATTGGAATGGAATGTCCTCTAATGGAATGGATTCGAGTGGAATGGAATCGAAT
ATAATGGAGTCGAATGAAATGGAATTGAAAGGAATGGGATCGAATACAATGGAATATACT
GGAATGTAATGTAATGAACTCGAATGTAATTGACTGGAATGGAATGTACATGAATGGAAT
GTAATCGAATGGAAAGTAATCCAATGTAATAGAATCTAATGCAACAAAATCGACTCAGAC
AGAGTAGAATGTAATGGAATGGAGTGCAGTGCAATGGAATGGAATGGAATGGAATGCAAT
GGAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGGGATGGAATGGAAT
GGAAT

In [25]:
17282478-17281497

981

In [24]:
17279312-17279311

1

In [23]:
#194	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	Yqhet_3k1bp	2	3119	17276196	17279311
#197	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ18_Yq	282	2862	17279911	17282478
#199	chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011	DYZ18_Yq	289	2862	17283081	17285642

with open('/home/mark/Desktop/HG02011_Transition.fasta', 'a+') as file:
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:17279311-17282478"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:17276196-17279311"))
    file.write(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:17279911-17282478"))
file.close()


In [42]:
HG02011_df[HG02011_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_18150388-18150679,0,2214,['44615 372 23.9 3.5 3.9 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18152810-18153098,0,2411,['44618 414 24.2 3.9 3.7 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18155225-18155516,0,2411,['44621 410 24.2 3.9 3.8 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18157634-18157921,0,2402,['44624 376 27.0 2.9 3.9 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18160046-18160334,0,2402,['44627 354 20.9 8.6 3.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18162462-18162752,0,2411,['44630 387 18.8 9.1 2.8 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18164877-18165164,0,2409,['44633 425 23.8 3.7 3.9 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_18167293-18167555,0,3404,['44636 295 27.2 5.8 3.0 chrY.09-24.01.HET1_ce...,3000-3999,#4169e1
+_AluY_18169723-18170014,0,2399,['44639 3914 10.3 0.0 0.0 chrY.09-24.01.HET1_c...,2000-2999,#4169e1


In [47]:
HG02011_df.loc['C_AluY_18164877-18165164']['Group']

"['44633 425 23.8 3.7 3.9 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18163327 18164876 22459284 False (TATATAA)n Simple_repeat 1 1547 0 35714 False', '44634 1345 19.8 0.0 5.5 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18164877 18165164 22458996 True AluY SINE/Alu 12 289 17 35715 False', '44635 4074 9.2 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18165168 18165735 22458425 True HSATI Satellite 0 568 1 35716 False']"

In [45]:
HG02011_df.loc['C_AluY_18167293-18167555']['Group']

"['44636 295 27.2 5.8 3.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18165744 18167290 22456870 False (ATATATC)n Simple_repeat 1 1588 0 35717 False', '44637 1226 20.6 0.0 6.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18167293 18167555 22456605 True AluY SINE/Alu 12 289 42 35718 False', '44638 419 24.2 3.9 3.5 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18167597 18169147 22455013 False (ATTATAT)n Simple_repeat 1 1556 0 35719 False']"

In [46]:
HG02011_df.loc['+_AluY_18169723-18170014']['Group']

"['44639 3914 10.3 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18169161 18169723 22454437 False HSATI Satellite 6 568 0 35720 False', '44640 1403 19.2 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18169723 18170014 22454146 False AluY SINE/Alu 13 289 12 35721 True', '44641 388 19.0 8.9 2.9 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18170016 18171559 22452601 False (TA)n Simple_repeat 1 1634 0 35722 False']"

In [50]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18165744-18169147"))


In [ ]:
################################### 2nd breakpoint #########################################3

In [43]:
HG02011_df[HG02011_df['RepeatBlock']==1]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_18901313-18901604,1,2412,['45180 4013 9.9 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_18903732-18904023,1,2413,['45183 3920 10.2 0.0 0.0 chrY.09-24.01.HET1_c...,2000-2999,#228B22
+_AluY_18906152-18906442,1,2412,['45186 4008 9.5 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_18908571-18908862,1,2413,['45189 3990 9.7 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#228B22
+_AluY_18910991-18911282,1,2424,['45192 3932 10.0 0.0 0.0 chrY.09-24.01.HET1_c...,2000-2999,#228B22
+_AluY_18913422-18913713,1,2411,['45195 3949 9.9 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_18915840-18916131,1,2411,['45198 3983 9.9 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_18918258-18918549,1,2411,['45201 4046 9.3 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_18920676-18920967,1,2411,['45204 3926 10.4 0.0 0.0 chrY.09-24.01.HET1_c...,2000-2999,#4169e1


In [44]:
HG02011_df[HG02011_df['RepeatBlock']==2]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_19050063-19050354,2,1546,['45322 174 24.1 4.1 3.9 chrY.09-24.01.HET1_ce...,1000-1999,#4169e1
C_AluY_19052479-19052771,2,2410,['45325 418 24.5 3.5 3.8 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19054894-19055185,2,2407,['45328 418 24.0 4.0 3.6 chrY.09-24.01.HET1_ce...,2000-2999,#228B22
C_AluY_19057312-19057603,2,2364,['45331 394 25.4 2.9 4.5 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19059669-19059960,2,2401,['45334 410 24.2 3.5 4.1 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19062092-19062381,2,2414,['45337 405 24.7 4.0 3.5 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19064510-19064801,2,2413,['45340 354 20.3 9.0 3.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19066924-19067215,2,2407,['45343 420 24.5 3.5 3.6 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_19069339-19069630,2,2408,['45346 408 24.1 4.1 3.7 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1


In [51]:
HG02011_df.loc['+_AluY_18944850-18945141']['Group']

"['45236 3883 10.7 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18944283 18944850 21679310 False HSATI Satellite 1 568 0 36263 False', '45237 1455 18.5 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18944850 18945141 21679019 False AluY SINE/Alu 13 289 12 36264 True', '45238 383 25.8 3.1 4.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 18945144 18945179 21678981 False (ATATATA)n Simple_repeat 1 1533 0 36265 False']"

In [57]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:18944850-18946179"))


In [52]:
HG02011_df.loc['C_AluY_19050063-19050354']['Group']

"['45322 174 24.1 4.1 3.9 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 19049376 19050060 21574100 False (TATATAT)n Simple_repeat 1 686 0 36346 False', '45323 1327 21.0 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 19050063 19050354 21573806 True AluY SINE/Alu 12 289 13 36347 True', '45324 3940 10.4 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 19050354 19050921 21573239 True HSATI Satellite 0 568 1 36348 False']"

In [56]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:19048376-19050354"))


In [ ]:
################################### 3rd Breakpoint ##########################################

In [149]:
HG02011_df[HG02011_df['RepeatBlock']==24]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_38998455-38998746,24,1609,['61395 185 24.6 4.0 4.0 chrY.09-24.01.HET1_ce...,1000-1999,#4169e1
C_AluY_39000877-39001167,24,2414,['61398 378 25.6 4.0 3.4 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39003298-39003589,24,2415,['61403 359 26.4 3.1 4.3 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39005717-39006008,24,2412,['61408 397 24.2 4.2 3.5 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39008137-39008428,24,2413,['61413 428 23.8 3.7 3.7 chrY.09-24.01.HET1_ce...,2000-2999,#228B22
C_AluY_39010553-39010844,24,2409,['61416 430 23.9 3.8 3.5 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39012950-39013241,24,2390,['61419 404 24.1 3.7 4.1 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39015363-39015654,24,2406,['61422 417 25.0 3.3 3.6 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39017923-39018214,24,2553,['61425 421 18.8 9.2 3.0 chrY.09-24.01.HET1_ce...,2000-2999,#228B22


In [150]:
HG02011_df.loc['C_AluY_39032428-39032719']['Group']

"['61443 424 23.6 3.7 4.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39030877 39032427 1591733 False (TATATAA)n Simple_repeat 1 1547 0 52098 False', '61444 1409 18.8 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39032428 39032719 1591441 True AluY SINE/Alu 12 289 13 52099 True', '61445 4003 9.3 0.2 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39032719 39033285 1590875 True HSATI Satellite 0 568 1 52100 False']"

In [151]:
HG02011_df.loc['+_AluY_39033956-39034234']['Group']

"['61446 156 19.4 9.6 2.8 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39033286 39033952 1590208 False (AT)n Simple_repeat 1 711 0 52101 False', '61447 1345 19.4 0.0 5.7 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39033956 39034234 1589926 False AluY SINE/Alu 26 289 12 52102 False', '61448 385 19.3 9.1 2.7 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39034236 39035783 1588377 False (TA)n Simple_repeat 1 1644 0 52103 False']"

In [152]:
HG02011_df.loc['+_AluY_39036358-39036649']['Group']

"['61449 3956 10.0 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39035791 39036358 1587802 False HSATI Satellite 1 568 0 52104 False', '61450 1390 19.2 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39036358 39036649 1587511 False AluY SINE/Alu 13 289 12 52105 True', '61451 403 18.7 9.1 2.8 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39036651 39038208 1585952 False (TA)n Simple_repeat 1 1653 0 52106 False']"

In [153]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:39032719-39036358"))


>chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:39032719-39036358
AAGATCCATATTTTTTAAATGAAAAAAAATTTCAAAGTACTCTGCTTGGTACAATAATCA
AATGTATAAACTGAGGAATAAAACATAACCATGAAACCTACTTATAACTGCATACGGAAA
ATACAGAGGATAATTTTTTAAATAACATATTTTGAAAGCCTTAACTAGTAATTTGAAGAG
TTGGCATTTGAAAGGCCAGTATGAATATACCTTCAAAGCAGCAACACAGGTTCCCCATGA
GAAAAAGCAAACTCAGGGAAAATAAAACCACAAAGGTTCAATCTCTTCTGACCTTTGAAA
GACACAGCACAGACAGTGGTCCTTAGGACGAAGAGCAGGAGACCCCTAATTCCGTCACCA
TGGCGATAGGGCATAAACATTGCTGGGTGAGGGCACAATCCACATTGTGAGGTCCAACTG
CTGCCAGGAAGACAGGAGTGCTTTTACATAGACCGAAAGGTCATTGAAGGCTCAGAGTTT
TGTTTCAAGAACTGAATCCCAAGCCCACACGTTATTAGGCTGCGCTTCTTAAAACAAGTT
ATGAGATGGGAAAAAGGGCACCCACAAATATACGTATACATTATGTATATAATAGTATAC
AGTATCATATATATAATATATATAACCTATGTATAATATTGTTTTACATCCTGCATCTTA
TATTATATATATTGTATAAAATATAATAATTGATAATATATAAGTTTTATATTTATACAA
TATGACATATAGTATATATTATAATTTATGACATATATTATGTATGTTATATATCATATA
TAGAATATATTATACGGTTAATATACGTATTAAATATGAATTATACACTATATATATTAT
ATATAATATATCTGTTATATATAATACTCATTATGCATAATATATATATTATGTCTATGT
TATAT

In [154]:
################################################ 4th Breakpoint ###############################

In [155]:
HG02011_df[HG02011_df['RepeatBlock']==25]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_39519411-39519702,25,2380,['61780 3925 10.4 0.0 0.0 chrY.09-24.01.HET1_c...,2000-2999,#4169e1
+_AluY_39521798-39522089,25,3695,['61783 4017 9.3 0.0 0.2 chrY.09-24.01.HET1_ce...,3000-3999,#4169e1
+_AluY_39525499-39525790,25,2407,['61786 4005 9.7 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_39527912-39528203,25,2410,['61789 3839 10.8 0.2 0.0 chrY.09-24.01.HET1_c...,2000-2999,#228B22
+_AluY_39530330-39530621,25,2413,['61792 3994 9.7 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_39532750-39533041,25,2413,['61795 4020 9.5 0.0 0.0 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
+_AluY_39535170-39535461,25,1750,['61800 4047 9.3 0.0 0.0 chrY.09-24.01.HET1_ce...,1000-1999,#228B22
C_AluY_39538490-39538781,25,2409,['61804 404 25.0 3.5 3.8 chrY.09-24.01.HET1_ce...,2000-2999,#4169e1
C_AluY_39540910-39541201,25,2413,['61807 410 24.9 3.4 3.9 chrY.09-24.01.HET1_ce...,2000-2999,#228B22


In [158]:
HG02011_df.loc['+_AluY_39532750-39533041']['Group']

"['61795 4020 9.5 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39532183 39532750 1091410 False HSATI Satellite 1 568 0 52435 False', '61796 1445 18.1 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39532750 39533041 1091119 False AluY SINE/Alu 13 289 12 52436 True', '61797 367 19.3 9.3 2.7 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39533043 39533894 1090266 False (TA)n Simple_repeat 1 897 0 52437 False', '61798 27 8.3 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39533895 39533933 1090227 False (ATATATC)n Simple_repeat 1 39 0 52438 False', '61799 367 19.3 9.3 2.7 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39533934 39534595 1089565 False (TA)n Simple_repeat 2 715 0 52439 False']"

In [156]:
HG02011_df.loc['+_AluY_39535170-39535461']['Group']

"['61800 4047 9.3 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39534603 39535170 1088990 False HSATI Satellite 1 568 0 52440 False', '61801 1409 18.8 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39535170 39535461 1088699 False AluY SINE/Alu 13 289 12 52441 True', '61802 222 19.2 8.1 3.7 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39535463 39536352 1087808 False (TA)n Simple_repeat 1 928 0 52442 False']"

In [157]:
HG02011_df.loc['C_AluY_39538490-39538781']['Group']

"['61804 404 25.0 3.5 3.8 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39536940 39538489 1085671 False (TATATAA)n Simple_repeat 1 1547 0 52444 False', '61805 1439 18.5 0.0 5.4 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39538490 39538781 1085379 True AluY SINE/Alu 12 289 13 52445 True', '61806 3985 9.7 0.0 0.0 chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011 39538781 39539348 1084812 True HSATI Satellite 0 568 1 52446 False']"

In [160]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG02011.HIFIRW.ONTUL.na.chrY.fasta', "chrY.09-24.01.HET1_centro-PAR2.FW.unassigned-0000585.HG02011:39535170-39538781"))



# HG01106

In [180]:
HG01106_df[HG01106_df['RepeatBlock']==37]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_36762197-36762488,37,1609,['79710 183 19.4 8.7 3.2 chrY.21-24.01.AMPL7-P...,1000-1999,#FF033E
C_AluY_36764619-36764909,37,2414,['79713 373 25.8 4.0 3.4 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
C_AluY_36767040-36767331,37,2415,['79718 366 19.4 9.4 2.7 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
C_AluY_36769600-36769891,37,2553,['79723 474 23.8 3.2 4.1 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
C_AluY_36772018-36772309,37,2411,['79726 395 18.7 9.1 2.9 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
C_AluY_36774435-36774726,37,2410,['79729 386 18.9 9.1 3.0 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
C_AluY_36776853-36777144,37,2411,['79732 420 24.2 3.5 3.9 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
C_AluY_36779269-36779560,37,2408,['79735 426 23.7 3.9 3.7 chrY.21-24.01.AMPL7-P...,2000-2999,#F6C324
C_AluY_36781685-36781976,37,2410,['79738 382 24.8 3.7 4.0 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E


In [182]:
HG01106_df.loc['C_AluY_36796202-36796491']['Group']

"['79758 394 26.1 3.0 4.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36794645 36796199 1403446 False (TATATAA)n Simple_repeat 1 1541 0 49509 False', '79759 1432 18.6 0.0 5.5 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36796202 36796491 1403154 True AluY SINE/Alu 14 287 13 49510 True', '79760 4043 9.3 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36796491 36797058 1402587 True HSATI Satellite 0 568 1 49511 False']"

In [183]:
HG01106_df.loc['+_AluY_36797729-36798007']['Group']

"['79761 157 19.3 9.8 2.7 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36797059 36797725 1401920 False (AT)n Simple_repeat 1 713 0 49512 False', '79762 1345 19.4 0.0 5.7 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36797729 36798007 1401638 False AluY SINE/Alu 26 289 12 49513 False', '79763 387 19.2 9.1 2.7 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36798009 36799558 1400087 False (TA)n Simple_repeat 1 1646 0 49514 False']"

In [184]:
HG01106_df.loc['+_AluY_36800133-36800424']['Group']

"['79764 3934 10.2 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36799566 36800133 1399512 False HSATI Satellite 1 568 0 49515 False', '79765 1390 19.2 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36800133 36800424 1399221 False AluY SINE/Alu 13 289 12 49516 True', '79766 401 18.8 9.1 2.8 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 36800426 36801983 1397662 False (TA)n Simple_repeat 1 1653 0 49517 False']"

In [185]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01106.HIFIRW.ONTUL.na.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106:36797058-36799566"))


>chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106:36797058-36799566
AATATACGTATACATTATGTATATAATAGTATACAGTATCATATATATAATATATATAAC
CTATGTATAATATTGTTTTACATCCTGCATCTTATATTATATATATTGTATAAAATATAA
TAATTGATAATATATAAGTTTTATATTTATACAATATGACATATAGTATATATTATAATT
TATGACATATATTATGTATGTTATATATCATATATAGAATATATTATACGGTTAATATAC
GTATTAAATATGAATTATACATTATATATATTATATATAATATATCTGTTATATATAATA
CTCATTATGCATAATATATATATTATGTCTATGTTATATATAGTATAAAACAACATATAA
TAATATATAATTGCATATATATAATGCGACATATAATATATATTATATTTTATGACATAT
GGAGTATATATTATATATTATATATAATACGTTGTATGTATAATACCTATATTAAATATA
TATTATATGACTATGTTATTAATTACATGTCACATATGTTATATATTATATATTTTACAT
AGAATATACCGGTTACATACAATATATACTATGTTACCTATAATGTATAATTTATTACAT
GTAAAATAAAGAATGCAATATATTATGTATAATTTATAATATACAATATGTGGAATGTGA
TATATATAACATAATCCCAGCAAATTTGGAGGCCAAGGCAAGGGGATCACAAGGTGAAGA
GATCAAGACCATCCTGGCCAATACAGTGAAACGCTGTCACTACTAAAAATCCAAAAAATT
AGCTAGGCATGGTGGCAGCTGGTGTAGTCCCAGTGTGGTGTAGTCCCAGCTACCTGGGAG
GCTGAGGGAGGACAATCGCTTGGACCCATGAGGCAGATGTTGCAGTGAGCAAAGATGGTG
CCATTGTACCC

In [186]:
################### 4th Breakpoint ######################################

In [187]:
HG01106_df[HG01106_df['RepeatBlock']==38]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_37130957-37131248,38,2380,['80011 3942 10.2 0.0 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#FF033E
+_AluY_37133344-37133635,38,3700,['80014 4016 9.3 0.0 0.2 chrY.21-24.01.AMPL7-P...,3000-3999,#FF033E
+_AluY_37137049-37137340,38,2410,['80017 3839 10.8 0.2 0.0 chrY.21-24.01.AMPL7-...,2000-2999,#F6C324
+_AluY_37139467-37139758,38,2413,['80020 3994 9.7 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
+_AluY_37141887-37142178,38,2413,['80023 4020 9.5 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
+_AluY_37144307-37144598,38,2413,['80028 4008 9.5 0.0 0.0 chrY.21-24.01.AMPL7-P...,2000-2999,#4169e1
+_AluY_37146728-37147019,38,1751,['80031 3894 10.4 0.0 0.2 chrY.21-24.01.AMPL7-...,1000-1999,#F6C324
C_AluY_37150048-37150339,38,2409,['80035 404 25.0 3.5 3.8 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E
C_AluY_37152467-37152758,38,2412,['80038 416 24.4 4.0 3.4 chrY.21-24.01.AMPL7-P...,2000-2999,#FF033E


In [188]:
HG01106_df.loc['+_AluY_37144307-37144598']['Group']

"['80028 4008 9.5 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37143740 37144307 1055338 False HSATI Satellite 1 568 0 49767 False', '80029 1457 18.5 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37144307 37144598 1055047 False AluY SINE/Alu 13 289 12 49768 True', '80030 415 24.7 3.9 3.3 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37144599 37146152 1053493 False (TTATATA)n Simple_repeat 1 1563 0 49769 False']"

In [189]:
HG01106_df.loc['+_AluY_37146728-37147019']['Group']

"['80031 3894 10.4 0.0 0.2 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37146160 37146728 1052917 False HSATI Satellite 1 568 0 49770 False', '80032 1468 18.8 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37146728 37147019 1052626 False AluY SINE/Alu 13 289 12 49771 True', '80033 231 18.6 8.1 3.7 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37147021 37147910 1051735 False (TA)n Simple_repeat 1 928 0 49772 False']"

In [190]:
HG01106_df.loc['C_AluY_37150048-37150339']['Group']

"['80035 404 25.0 3.5 3.8 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37148498 37150047 1049598 False (TATATAA)n Simple_repeat 1 1547 0 49774 False', '80036 1439 18.5 0.0 5.4 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37150048 37150339 1049306 True AluY SINE/Alu 12 289 13 49775 True', '80037 3985 9.7 0.0 0.0 chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106 37150339 37150906 1048739 True HSATI Satellite 0 568 1 49776 False']"

In [191]:
print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly/FinalGenomes/HG01106.HIFIRW.ONTUL.na.chrY.fasta', "chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106:37146728-37150339"))


>chrY.21-24.01.AMPL7-PAR2.FW.unassigned-0000976.HG01106:37146728-37150339
TGGCTCAGGCCTGTAATCCCAGCAAATTTGGAGGCCAAGGCAAGGGGATCACAAGGTGAA
GAGATCAAGACCATCCTGGCCAATACAGTGAAACGCTGTCACTACTAAAAATCCAAAAAA
TTAGCTAGGCATGGTGGCAGCTGCTGTAGTCCCAGTGTGGTGTAGTCCCAGCTACCTGGG
AGGCTGAGGGAGGACAATCGCTTGGACCCGTGAGGCAGATGTTGCAGTGAGCAAAGATGG
CGCCATTGTACCCCACCCTGTGCAACAGGACAAGATTCTGTCTAAAAAAAAATTATATAT
GATATATATTACATGTTATGTGCTATGCCTTATATGTAACATATAATCTCATATATTTTA
TATGTCATAGTATATAATATTTATCCTGTGTCATCTTATATATACTATATATATTACATA
ACATATATAATATATGATACATATTATACATCTTATATATGAAATTATATAATTATAGAT
AACATAATACACATTTATATGTATTATGATATATAACAAAGGTAGTATATCATATATGAT
ATGTAATATATCTATTTGTAGTACATATTATAATTTTTATATATTATAATTTATATAATG
ATATAATTCTTTATAGAATATAAATAATTATATATGATTATATAATTCTATATATTATAA
ATGAAAATATGTAGAATTACATGTTTATCTAGTTATATGTATAATAATATAATTATATAT
ATAGTTATATATATCACAGAATAGACAACATACATATTATATAATGTATAATATATATTA
TAGAATATATGATACATGATATATATGATATTTAATATAACATAATATAATCTATATTAT
AATATTATGTATGTTACTTATATTGGGTGATATGTAATATATATTATGTAATATGAAATA
GTATAATATAT

# HG01890

In [7]:
HG01890_df[HG01890_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_29825231-29825519,0,2216,['45510 373 24.0 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29827646-29827937,0,2411,['45513 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29830064-29830355,0,2411,['45516 405 24.4 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29832473-29832761,0,2399,['45519 363 27.6 2.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29834883-29835171,0,2403,['45522 410 24.2 4.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29837299-29837589,0,2411,['45525 378 19.1 9.1 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29839713-29840000,0,2408,['45529 397 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29842134-29842422,0,2411,['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_29846411-29846702,0,2406,['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22


In [11]:
HG01890_df.loc['C_AluY_29839713-29840000']['Group']

"['45529 397 18.8 9.1 2.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29838164 29839711 16568193 False (TA)n Simple_repeat 1 1644 0 36531 False', '45530 1372 19.5 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29839713 29840000 16567904 True AluY SINE/Alu 12 289 17 36532 False', '45531 4089 9.0 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29840004 29840571 16567333 True HSATI Satellite 0 568 1 36533 False']"

In [8]:
HG01890_df.loc['C_AluY_29842134-29842422']['Group']

"['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29840579 29842129 16565775 False (TA)n Simple_repeat 1 1634 0 36534 False', '45533 1377 19.4 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29842134 29842422 16565482 True AluY SINE/Alu 15 286 13 36535 True', '45534 3909 10.6 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29842422 29842989 16564915 True HSATI Satellite 0 568 1 36536 False']"

In [12]:
29845844-29842989

2855

In [9]:
HG01890_df.loc['+_AluY_29846411-29846702']['Group']

"['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29845844 29846411 16561493 False HSATI Satellite 1 568 0 36538 False', '45537 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846411 29846702 16561202 False AluY SINE/Alu 13 289 12 36539 True', '45538 382 19.4 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846704 29848249 16559655 False (TA)n Simple_repeat 1 1632 0 36540 False']"

In [10]:
HG01890_df.loc['+_AluY_29848824-29849115']['Group']

"['45539 3962 10.0 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29848257 29848824 16559080 False HSATI Satellite 1 568 0 36541 False', '45540 1397 19.9 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29848824 29849115 16558789 False AluY SINE/Alu 13 289 12 36542 True', '45541 381 19.0 8.8 3.2 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29849117 29850656 16557248 False (TA)n Simple_repeat 1 1624 0 36543 False']"

In [14]:
#print(pysam.faidx('/home/mark/Desktop/chromosome_y/assemblies/assembly_pysam/HG01890.chrY.fasta', "chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842134-29846702"))


In [5]:
import ast
HG01890_df['Start']='temp'
HG01890_df['End']='temp'
for row in HG01890_df.index:
    HG01890_df.at[row,'Start']= ast.literal_eval(HG01890_df.at[row,'Group'])[0].split(" ")[6]
    HG01890_df.at[row,'End']= ast.literal_eval(HG01890_df.at[row,'Group'])[-1].split(" ")[7]
HG01890_df

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End
Alu,,,,,,,
C_AluY_29825231-29825519,0,2216,['45510 373 24.0 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,29823871,29826086
C_AluY_29827646-29827937,0,2411,['45513 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,29826094,29828504
C_AluY_29830064-29830355,0,2411,['45516 405 24.4 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,29828512,29830922
C_AluY_29832473-29832761,0,2399,['45519 363 27.6 2.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,29830930,29833328
C_AluY_29834883-29835171,0,2403,['45522 410 24.2 4.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,29833336,29835738
...,...,...,...,...,...,...,...
+_AluSx_46159487-46159815,20,507,['60051 2110 13.0 0.7 0.0 chrY.01-24.01.PAR1-P...,<999,GraveYard,46159487,46159993
+_AluJb_46160943-46161093,20,1012,['60053 18 21.8 0.0 0.0 chrY.01-24.01.PAR1-PAR...,1000-1999,GraveYard,46160082,46161093
+_AluSp_46161094-46161391,20,298,['60055 2100 12.8 0.7 0.3 chrY.01-24.01.PAR1-P...,<999,GraveYard,46161094,46161391


In [ ]:
##################################### HG01890 1st Breakpoint ####################################

In [ ]:
####### 1st Breakpoint happens around 1790 in between the two subunits within a TA simple repeat region #########

In [11]:
HG01890_df.loc['C_AluY_29842134-29842422']['Group']

"['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29840579 29842129 16565775 False (TA)n Simple_repeat 1 1634 0 36534 False', '45533 1377 19.4 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29842134 29842422 16565482 True AluY SINE/Alu 15 286 13 36535 True', '45534 3909 10.6 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29842422 29842989 16564915 True HSATI Satellite 0 568 1 36536 False']"

In [12]:
HG01890_df.loc['+_AluY_29846411-29846702']['Group']

"['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29845844 29846411 16561493 False HSATI Satellite 1 568 0 36538 False', '45537 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846411 29846702 16561202 False AluY SINE/Alu 13 289 12 36539 True', '45538 382 19.4 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29846704 29848249 16559655 False (TA)n Simple_repeat 1 1632 0 36540 False']"

In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842422-29845844
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29842989-29846411

In [13]:
29845844-29842989

2855

In [ ]:
################# 2nd Breakpoint ######################

In [20]:
#2nd Breakpoint seems to be at the end of the simple repeat of +_AluY_29856054-29856345 (either simple repeat or HSATII satellite according to repeatmasker)

In [15]:
HG01890_df[HG01890_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_29825231-29825519,0,2216,['45510 373 24.0 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29827646-29827937,0,2411,['45513 407 24.3 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29830064-29830355,0,2411,['45516 405 24.4 3.9 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29832473-29832761,0,2399,['45519 363 27.6 2.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29834883-29835171,0,2403,['45522 410 24.2 4.6 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29837299-29837589,0,2411,['45525 378 19.1 9.1 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29839713-29840000,0,2408,['45529 397 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_29842134-29842422,0,2411,['45532 366 20.5 8.5 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
+_AluY_29846411-29846702,0,2406,['45536 3978 9.9 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22


In [17]:
HG01890_df.loc['+_AluY_29853656-29853948']['Group']

"['45545 3910 10.1 0.2 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29853090 29853656 16554248 False HSATI Satellite 1 568 0 36547 False', '45546 1239 21.5 0.0 6.2 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29853656 29853948 16553956 False AluY SINE/Alu 13 288 13 36548 True', '45547 425 24.4 3.1 3.8 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29853949 29855479 16552425 False (TTATATA)n Simple_repeat 1 1520 0 36549 False']"

In [16]:
HG01890_df.loc['+_AluY_29856054-29856345']['Group']

"['45548 3947 10.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29855487 29856054 16551850 False HSATI Satellite 1 568 0 36550 False', '45549 1293 20.0 0.0 5.8 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29856054 29856345 16551559 False AluY SINE/Alu 13 288 13 36551 True', '45550 177 19.3 8.3 3.1 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 29856347 29857041 16550863 False (TA)n Simple_repeat 1 730 0 36552 False']"

In [18]:
HG01890_df.loc['C_AluY_30204878-30205169']['Group']

"['45750 425 23.9 3.6 3.8 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 30203322 30204868 16203036 False (ATAATAT)n Simple_repeat 1 1543 0 36739 False', '45751 1455 18.5 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 30204878 30205169 16202735 True AluY SINE/Alu 12 289 13 36740 True', '45752 3932 10.4 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 30205169 30205736 16202168 True HSATI Satellite 0 568 1 36741 False']"

In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:29856347-29858041 #+_AluY_29856054-29856345 simple repeat +1000 into spacer block
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:30202322-30204868

In [ ]:
#3rd Breakpoint

In [25]:
#for x in HG01890_df[HG01890_df['RepeatBlock']==14].index:
#    print(x)

In [24]:
HG01890_df[HG01890_df['RepeatBlock']==14].loc[['C_AluY_44443000-44443289','C_AluY_44445415-44445706','+_AluY_44446944-44447124','+_AluY_44448773-44449064']]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
C_AluY_44443000-44443289,14,2412,['58450 376 19.1 9.0 3.2 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_44445415-44445706,14,2411,['58453 408 24.2 4.2 3.6 chrY.01-24.01.PAR1-PA...,2000-2999,#4169e1
+_AluY_44446944-44447124,14,1917,['58456 168 25.4 4.6 2.5 chrY.01-24.01.PAR1-PA...,1000-1999,#FF033E
+_AluY_44448773-44449064,14,2405,['58459 3974 10.2 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#228B22


In [26]:
HG01890_df.loc['C_AluY_44445415-44445706']['Group']

"['58453 408 24.2 4.2 3.6 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44443863 44445414 1962490 False (TATATAA)n Simple_repeat 1 1561 0 49150 False', '58454 1441 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44445415 44445706 1962198 True AluY SINE/Alu 12 289 13 49151 True', '58455 4005 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44445706 44446273 1961631 True HSATI Satellite 0 568 1 49152 False']"

In [27]:
HG01890_df.loc['+_AluY_44446944-44447124']['Group']

"['58456 168 25.4 4.6 2.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44446281 44446938 1960966 False (TATATAA)n Simple_repeat 1 671 0 49153 False', '58457 865 17.0 0.0 9.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44446944 44447124 1960780 False AluY SINE/Alu 26 191 110 49154 False', '58458 298 25.7 2.5 3.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44447131 44448197 1959707 False (TATATAT)n Simple_repeat 1 1057 0 49155 False']"

In [28]:
HG01890_df.loc['+_AluY_44448773-44449064']['Group']

"['58459 3974 10.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44448206 44448773 1959131 False HSATI Satellite 1 568 0 49156 False', '58460 1381 19.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44448773 44449064 1958840 False AluY SINE/Alu 13 289 12 49157 True', '58461 410 24.0 3.3 4.5 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 44449065 44450610 1957294 False (TTATATA)n Simple_repeat 1 1527 0 49158 False']"

In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:44446273-44448206  #Looks like right at the boundary of simple repeat meets head of sense Alu

In [ ]:
################# 4th Breakpoint #####################

In [31]:
HG01890_df[HG01890_df['RepeatBlock']==15]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster
Alu,,,,,
+_AluY_45031551-45031842,15,2382,['58925 3950 10.2 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#228B22
+_AluY_45033940-45034231,15,3696,['58928 4015 9.3 0.0 0.2 chrY.01-24.01.PAR1-PA...,3000-3999,#228B22
+_AluY_45037642-45037933,15,2409,['58931 3991 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#4169e1
+_AluY_45040059-45040350,15,1751,['58934 3869 10.6 0.0 0.2 chrY.01-24.01.PAR1-P...,1000-1999,#4169e1
C_AluY_45043380-45043671,15,2410,['58938 410 24.8 3.3 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_45045796-45046087,15,2409,['58941 412 25.2 3.2 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_45048216-45048507,15,2413,['58944 403 25.3 3.6 3.6 chrY.01-24.01.PAR1-PA...,2000-2999,#4169e1
C_AluY_45050633-45050924,15,2410,['58947 407 24.7 4.1 3.4 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22
C_AluY_45053052-45053343,15,2412,['58950 406 25.1 3.4 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#4169e1


In [32]:
HG01890_df.loc['+_AluY_45037642-45037933']['Group']

"['58931 3991 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45037075 45037642 1370262 False HSATI Satellite 1 568 0 49611 False', '58932 1450 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45037642 45037933 1369971 False AluY SINE/Alu 13 289 12 49612 True', '58933 416 24.8 3.3 3.8 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45037934 45039483 1368421 False (TTATATA)n Simple_repeat 1 1541 0 49613 False']"

In [35]:
45039491-45039483

8

In [33]:
HG01890_df.loc['+_AluY_45040059-45040350']['Group']

"['58934 3869 10.6 0.0 0.2 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45039491 45040059 1367845 False HSATI Satellite 1 568 0 49614 False', '58935 1468 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45040059 45040350 1367554 False AluY SINE/Alu 13 289 12 49615 True', '58936 238 18.1 8.1 3.7 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45040352 45041241 1366663 False (TA)n Simple_repeat 1 928 0 49616 False']"

In [36]:
45041829-45041241

588

In [34]:
HG01890_df.loc['C_AluY_45043380-45043671']['Group']

"['58938 410 24.8 3.3 4.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45041829 45043379 1364525 False (TATATAA)n Simple_repeat 1 1540 0 49618 False', '58939 1439 18.5 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45043380 45043671 1364233 True AluY SINE/Alu 12 289 13 49619 True', '58940 3996 9.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890 45043671 45044238 1363666 True HSATI Satellite 0 568 1 49620 False']"

In [ ]:
chrY.01-24.01.PAR1-PAR2.RV.utig4-384.HG01890:45040350-45043380 #Seems to be between simple repeat and start of HSATI satelllite

# In Between

In [11]:
import ast
HC02666_df['Start']='temp'
HC02666_df['End']='temp'
for row in HC02666_df.index:
    HC02666_df.at[row,'Start']= ast.literal_eval(HC02666_df.at[row,'Group'])[0].split(" ")[6]
    HC02666_df.at[row,'End']= ast.literal_eval(HC02666_df.at[row,'Group'])[-1].split(" ")[7]
HC02666_df

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End
Alu,,,,,,,
C_AluY_30358347-30358634,0,2214,['45593 372 23.4 4.5 3.3 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30356992,30359205
C_AluY_30360759-30361047,0,2402,['45596 372 19.8 8.9 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30359213,30361614
C_AluY_30363169-30363457,0,2403,['45599 400 24.7 3.9 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30361622,30364024
C_AluY_30365580-30365868,0,2404,['45602 379 26.2 3.0 4.2 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30364032,30366435
C_AluY_30367996-30368286,0,2418,['45605 390 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30366436,30368853
...,...,...,...,...,...,...,...
+_AluSx_54438944-54439272,25,551,['65992 2114 13.0 0.7 0.0 chrY.01-24.01.PAR1-P...,<999,GraveYard,54438944,54439494
+_AluJb_54440400-54440550,25,1012,['65994 18 21.8 0.0 0.0 chrY.01-24.01.PAR1-PAR...,1000-1999,GraveYard,54439539,54440550
+_AluSp_54440551-54440848,25,298,['65996 2102 12.8 0.7 0.3 chrY.01-24.01.PAR1-P...,<999,GraveYard,54440551,54440848


In [14]:
HC02666_df[HC02666_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End
Alu,,,,,,,
C_AluY_30358347-30358634,0,2214,['45593 372 23.4 4.5 3.3 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30356992,30359205
C_AluY_30360759-30361047,0,2402,['45596 372 19.8 8.9 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30359213,30361614
C_AluY_30363169-30363457,0,2403,['45599 400 24.7 3.9 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30361622,30364024
C_AluY_30365580-30365868,0,2404,['45602 379 26.2 3.0 4.2 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30364032,30366435
C_AluY_30367996-30368286,0,2418,['45605 390 18.8 9.1 2.7 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30366436,30368853
C_AluY_30370411-30370698,0,2409,['45608 425 23.8 3.7 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30368861,30371269
C_AluY_30372829-30373091,0,1855,['45611 1214 21.0 0.0 6.0 chrY.01-24.01.PAR1-P...,1000-1999,#F6C324,30372829,30374683
+_AluY_30375259-30375550,0,2399,['45613 3890 10.5 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#F6C324,30374697,30377095
+_AluY_30377670-30377961,0,2406,['45616 4003 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,30377103,30379508


In [18]:
HC02666_df.loc['C_AluY_30370411-30370698']['Group']

"['45608 425 23.8 3.7 3.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30368861 30370410 24412047 False (TATATAA)n Simple_repeat 1 1547 0 36449 False', '45609 1345 19.8 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30370411 30370698 24411759 True AluY SINE/Alu 12 289 17 36450 False', '45610 4063 9.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30370702 30371269 24411188 True HSATI Satellite 0 568 1 36451 False']"

In [17]:
HC02666_df.loc['C_AluY_30372829-30373091']['Group']

"['45611 1214 21.0 0.0 6.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30372829 30373091 24409366 True AluY SINE/Alu 12 289 42 36452 False', '45612 422 24.1 3.9 3.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30373133 30374683 24407774 False (ATTATAT)n Simple_repeat 1 1556 0 36453 False']"

In [16]:
HC02666_df.loc['+_AluY_30375259-30375550']['Group']

"['45613 3890 10.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30374697 30375259 24407198 False HSATI Satellite 6 568 0 36454 False', '45614 1403 19.2 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375259 30375550 24406907 False AluY SINE/Alu 13 289 12 36455 True', '45615 388 19.0 8.9 2.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30375552 30377095 24405362 False (TA)n Simple_repeat 1 1634 0 36456 False']"

In [19]:
HC02666_df.loc['+_AluY_30377670-30377961']['Group']

"['45616 4003 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377103 30377670 24404787 False HSATI Satellite 1 568 0 36457 False', '45617 1417 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377670 30377961 24404496 False AluY SINE/Alu 13 289 12 36458 True', '45618 389 19.2 8.7 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30377963 30379508 24402949 False (TA)n Simple_repeat 1 1632 0 36459 False']"

In [ ]:
#+-1000
#Repeat Block 0
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30367861-30372269
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30371829-30375683
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30373697-30378095
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30376103-30380508

In [ ]:
#+-500
#Repeat Block 0
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372829-30375183
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30374197-30377095

In [ ]:
#No Overlap
#Repeat Block 0
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30368861-30371269
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30372829-30374683
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30374697-30377095

In [ ]:
#Drawing
#Repeat Block 0
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30368861-30371369
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30374597-30377095

# Second Inversion 

In [47]:
HC02666_df[HC02666_df['RepeatBlock']==1].loc['+_AluY_30951746-30952037']['Group']

"['46022 3950 9.7 0.2 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30951180 30951746 23830711 False HSATI Satellite 1 568 0 36825 False', '46023 1444 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30951746 30952037 23830420 False AluY SINE/Alu 13 289 12 36826 True', '46024 374 19.5 8.9 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30952039 30953591 23828866 False (TA)n Simple_repeat 1 1642 0 36827 False']"

In [52]:
30953599-30953591

8

In [48]:
HC02666_df[HC02666_df['RepeatBlock']==1].loc['+_AluY_30954166-30954457']['Group']

"['46025 3990 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30953599 30954166 23828291 False HSATI Satellite 1 568 0 36828 False', '46026 1459 18.1 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30954166 30954457 23828000 False AluY SINE/Alu 13 289 12 36829 True', '46027 393 18.8 9.0 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 30954459 30954914 23827543 False (TA)n Simple_repeat 1 1646 0 36830 False']"

In [ ]:
##########################################################################

In [ ]:
#Drawing
#Repeat Block 1-2
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30954459-30955914 (SIMPLE REPEAT BLOCK 1) actually long
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:31111344-31112344 ( BEFORE SIMPLE REPEAT AND SIMPLE REPEAT BLOCK 2)


In [58]:
31112344-31111344

1000

In [57]:
30954914-30954459

455

In [56]:
30955914-30954459

1455

In [ ]:
# Part2
#Repeat Block 1-2
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:30955414-30956914 (SIMPLE REPEAT BLOCK 1) actually long
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:31111344-31112344 ( BEFORE SIMPLE REPEAT AND SIMPLE REPEAT BLOCK 2)


In [ ]:
##########################################################################

In [50]:
HC02666_df[HC02666_df['RepeatBlock']==2].loc['C_AluY_31112347-31112638']['Group']

"['46145 179 23.7 4.1 3.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31111660 31112344 23670113 False (TATATAT)n Simple_repeat 1 686 0 36946 False', '46146 1355 20.3 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31112347 31112638 23669819 True AluY SINE/Alu 12 289 13 36947 True', '46147 3940 10.4 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31112638 31113205 23669252 True HSATI Satellite 0 568 1 36948 False']"

In [51]:
HC02666_df[HC02666_df['RepeatBlock']==2].loc['C_AluY_31114767-31115058']['Group']

"['46148 405 24.5 4.0 3.6 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31113213 31114766 23667691 False (TATATAA)n Simple_repeat 1 1561 0 36949 False', '46149 1381 18.3 1.0 6.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31114767 31115058 23667399 True AluY SINE/Alu 12 289 13 36950 True', '46150 3888 10.2 0.2 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 31115058 31115624 23666833 True HSATI Satellite 0 568 1 36951 False']"

# Third Inversion HC02666

In [29]:
tempList= ['C_AluY_52797337-52797628','C_AluY_52799760-52800049', 'C_AluY_52802178-52802469', '+_AluY_52803700-52803978', '+_AluY_52806106-52806397','+_AluY_52808523-52808814']

In [24]:
HC02666_df.loc['C_AluY_52799760-52800049']['Group']

"['64617 397 25.3 3.5 3.9 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52798203 52799757 1982700 False (TATATAA)n Simple_repeat 1 1548 0 54946 False', '64618 1458 17.9 0.0 5.5 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52799760 52800049 1982408 True AluY SINE/Alu 14 287 13 54947 True', '64619 4018 9.5 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52800049 52800616 1981841 True HSATI Satellite 0 568 1 54948 False']"

In [25]:
HC02666_df.loc['C_AluY_52802178-52802469']['Group']

"['64620 353 20.1 9.1 3.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52800624 52802176 1980281 False (TA)n Simple_repeat 1 1644 0 54949 False', '64621 1433 18.5 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52802178 52802469 1979988 True AluY SINE/Alu 12 289 13 54950 True', '64622 4000 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52802469 52803036 1979421 True HSATI Satellite 0 568 1 54951 False']"

In [32]:
52803044-52803036

8

In [26]:
HC02666_df.loc['+_AluY_52803700-52803978']['Group']

"['64623 167 25.0 4.2 3.2 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52803044 52803694 1978763 False (TATATAA)n Simple_repeat 1 657 0 54952 False', '64624 1365 18.6 0.0 5.7 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52803700 52803978 1978479 False AluY SINE/Alu 26 289 12 54953 False', '64625 410 24.2 3.9 3.7 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52803979 52805531 1976926 False (TTATATA)n Simple_repeat 1 1556 0 54954 False']"

In [35]:
52805539-52805531

8

In [27]:
HC02666_df.loc['+_AluY_52806106-52806397']['Group']

"['64626 3925 10.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52805539 52806106 1976351 False HSATI Satellite 1 568 0 54955 False', '64627 1399 19.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52806106 52806397 1976060 False AluY SINE/Alu 13 289 12 54956 True', '64628 399 24.4 4.1 3.7 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52806398 52807948 1974509 False (TTATATA)n Simple_repeat 1 1556 0 54957 False']"

In [31]:
HC02666_df.loc['+_AluY_52808523-52808814']['Group']

"['64629 3925 10.2 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52807956 52808523 1973934 False HSATI Satellite 1 568 0 54958 False', '64630 1399 19.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52808523 52808814 1973643 False AluY SINE/Alu 13 289 12 54959 True', '64631 410 24.4 4.1 3.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52808815 52808873 1973584 False (TTATATA)n Simple_repeat 1 1562 0 54960 False', '64632 401 23.9 4.3 3.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 52808874 52810365 1972092 False (TATATAT)n Simple_repeat 1 1505 0 54961 False']"

In [30]:
HC02666_df[HC02666_df['RepeatBlock']==19].loc[tempList]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End
Alu,,,,,,,
C_AluY_52797337-52797628,19,2413,['64614 368 19.7 9.0 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,52795783,52798195
C_AluY_52799760-52800049,19,2414,['64617 397 25.3 3.5 3.9 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,52798203,52800616
C_AluY_52802178-52802469,19,2413,['64620 353 20.1 9.1 3.0 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,52800624,52803036
+_AluY_52803700-52803978,19,2488,['64623 167 25.0 4.2 3.2 chrY.01-24.01.PAR1-PA...,2000-2999,#228B22,52803044,52805531
+_AluY_52806106-52806397,19,2410,['64626 3925 10.2 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#F6C324,52805539,52807948
+_AluY_52808523-52808814,19,2410,['64629 3925 10.2 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#F6C324,52807956,52810365


In [ ]:
#Drawing
#Repeat Block 19
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52802936-52804936 (last 100 bp + 2000 - C_AluY_52802178-52802469)
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52803639-52805639 (+_AluY_52806106-52806397)

In [ ]:
#Drawing 2 surrounding Alu
#Repeat Block 19
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52803044-52804079
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:52803594-52804531

## Block 20

In [36]:
HC02666_df[HC02666_df['RepeatBlock']==20]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End
Alu,,,,,,,
+_AluY_53441519-53441810,20,2382,['65281 3971 10.0 0.0 0.0 chrY.01-24.01.PAR1-P...,2000-2999,#F6C324,53440952,53443333
+_AluY_53443907-53444198,20,1750,['65284 3990 9.7 0.0 0.0 chrY.01-24.01.PAR1-PA...,1000-1999,#F6C324,53443340,53445089
C_AluY_53447227-53447518,20,2409,['65288 411 24.8 3.5 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,53445677,53448085
C_AluY_53449645-53449936,20,2411,['65291 407 24.4 3.7 4.0 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,53448093,53450503
C_AluY_53452064-53452355,20,2412,['65294 416 24.4 4.0 3.4 chrY.01-24.01.PAR1-PA...,2000-2999,#F6C324,53450511,53452922
C_AluY_53454484-53454775,20,2413,['65297 416 24.5 3.7 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E,53452930,53455342
C_AluY_53456904-53457195,20,2413,['65300 409 24.4 3.8 3.8 chrY.01-24.01.PAR1-PA...,2000-2999,#4169e1,53455350,53457762
C_AluY_53459322-53459613,20,2411,['65303 407 24.7 3.8 3.7 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E,53457770,53460180
C_AluY_53461742-53462033,20,2413,['65306 378 19.7 8.9 2.8 chrY.01-24.01.PAR1-PA...,2000-2999,#FF033E,53460188,53462600


In [39]:
HC02666_df.loc['+_AluY_53441519-53441810']['Group']

"['65281 3971 10.0 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53440952 53441519 1340938 False HSATI Satellite 1 568 0 55574 False', '65282 1405 18.8 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53441519 53441810 1340647 False AluY SINE/Alu 13 289 12 55575 True', '65283 446 24.2 3.1 3.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53441811 53443333 1339124 False (TTATATA)n Simple_repeat 1 1519 0 55576 False']"

In [42]:
53443340-53443333

7

In [38]:
HC02666_df.loc['+_AluY_53443907-53444198']['Group']

"['65284 3990 9.7 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53443340 53443907 1338550 False HSATI Satellite 1 568 0 55577 False', '65285 1426 19.6 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53443907 53444198 1338259 False AluY SINE/Alu 13 289 12 55578 True', '65286 230 19.0 8.0 3.6 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53444200 53445089 1337368 False (TA)n Simple_repeat 1 928 0 55579 False']"

In [41]:
53445677-53445089

588

In [40]:
HC02666_df.loc['C_AluY_53447227-53447518']['Group']

"['65288 411 24.8 3.5 3.8 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53445677 53447226 1335231 False (TATATAA)n Simple_repeat 1 1547 0 55581 False', '65289 1480 17.7 0.0 5.4 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53447227 53447518 1334939 True AluY SINE/Alu 12 289 13 55582 True', '65290 3959 9.9 0.0 0.0 chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666 53447518 53448085 1334372 True HSATI Satellite 0 568 1 55583 False']"

In [ ]:
#Drawing 
#Repeat Block 20
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53443907-53445089 (Alu-SR)
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53445677-53447518 (SR - ALU)

In [ ]:
#Drawing 
#Repeat Block 20
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53444200-53445689 
#chrY.01-24.01.PAR1-PAR2.RV.unassigned-0000749.HC02666:53445677-53447226 

# T2T Inversions

In [16]:
import ast
T2T_df['Start']='temp'
T2T_df['End']='temp'
for row in T2T_df.index:
    T2T_df.at[row,'Start']= ast.literal_eval(T2T_df.at[row,'Group'])[0].split(" ")[5]
    T2T_df.at[row,'End']= ast.literal_eval(T2T_df.at[row,'Group'])[-1].split(" ")[6]
T2T_df['SpaceBetween']=0
flag=0

for row in T2T_df.index:
    if flag==0:
        T2T_df.at[row,'SpaceBetween'] = 0
        previousRow = row
        flag+=1
    else:
        T2T_df.at[row,'SpaceBetween'] =  int(T2T_df.at[row,'Start'])-int(T2T_df.at[previousRow,'End'])
        previousRow=row
T2T_df

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End,SpaceBetween
Alu,,,,,,,,
C_AluY_28661430-28661721,0,2217,['373 24.5 3.2 3.8 chrY 28660073 28661429 (337...,2000-2999,#FF033E,28660073,28662289,0
C_AluY_28663852-28664140,0,2411,['414 24.2 3.9 3.7 chrY 28662297 28663849 (337...,2000-2999,#FF033E,28662297,28664707,8
C_AluY_28666267-28666558,0,2411,['407 24.3 3.9 3.8 chrY 28664715 28666266 (337...,2000-2999,#FF033E,28664715,28667125,8
C_AluY_28668676-28668963,0,2402,['381 26.8 2.9 3.9 chrY 28667133 28668673 (337...,2000-2999,#FF033E,28667133,28669534,8
C_AluY_28671087-28671377,0,2403,['358 20.7 8.6 2.8 chrY 28669542 28671048 (337...,2000-2999,#FF033E,28669542,28671944,8
...,...,...,...,...,...,...,...,...
+_AluSx_62119706-62120034,45,551,['2114 13.0 0.7 0.0 chrY 62119706 62120034 (33...,<999,GraveYard,62119706,62120256,18
+_AluJb_62121162-62121312,45,1012,['18 21.8 0.0 0.0 chrY 62120301 62120337 (3396...,1000-1999,GraveYard,62120301,62121312,45
+_AluSp_62121313-62121610,45,298,['2085 13.1 0.7 0.3 chrY 62121313 62121610 (33...,<999,GraveYard,62121313,62121610,1


## 1st Inversion Block 0

In [17]:
T2T_df[T2T_df['RepeatBlock']==0]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End,SpaceBetween
Alu,,,,,,,,
C_AluY_28661430-28661721,0,2217,['373 24.5 3.2 3.8 chrY 28660073 28661429 (337...,2000-2999,#FF033E,28660073,28662289,0
C_AluY_28663852-28664140,0,2411,['414 24.2 3.9 3.7 chrY 28662297 28663849 (337...,2000-2999,#FF033E,28662297,28664707,8
C_AluY_28666267-28666558,0,2411,['407 24.3 3.9 3.8 chrY 28664715 28666266 (337...,2000-2999,#FF033E,28664715,28667125,8
C_AluY_28668676-28668963,0,2402,['381 26.8 2.9 3.9 chrY 28667133 28668673 (337...,2000-2999,#FF033E,28667133,28669534,8
C_AluY_28671087-28671377,0,2403,['358 20.7 8.6 2.8 chrY 28669542 28671048 (337...,2000-2999,#FF033E,28669542,28671944,8
C_AluY_28673501-28673763,0,2430,['1228 20.2 0.0 6.0 chrY 28673501 28673763 (33...,2000-2999,#FF033E,28673501,28675930,1557
+_AluY_28675930-28676221,0,1839,['1403 19.2 0.0 5.4 chrY 28675930 28676221 (33...,1000-1999,#FF033E,28675930,28677768,0
+_AluY_28678343-28678634,0,2400,['4026 9.5 0.0 0.0 chrY 28677776 28678343 (337...,2000-2999,#FF033E,28677776,28680175,8
+_AluY_28680761-28681053,0,2408,['4022 9.5 0.0 0.0 chrY 28680194 28680761 (337...,2000-2999,#FF033E,28680194,28682601,19


In [10]:
T2T_df.loc['C_AluY_28671087-28671377']['Group']

"['358 20.7 8.6 2.8 chrY 28669542 28671048 (33788981) + (TA)n Simple_repeat 1 1592 (0) 36037', '1397 18.5 0.3 5.4 chrY 28671087 28671377 (33788652) C AluY SINE/Alu (12) 289 13 36038 *', '4088 9.0 0.0 0.0 chrY 28671377 28671944 (33788085) C HSATI Satellite (0) 568 1 36039']"

In [7]:
T2T_df.loc['C_AluY_28673501-28673763']['Group']

"['1228 20.2 0.0 6.0 chrY 28673501 28673763 (33786266) C AluY SINE/Alu (12) 289 42 36040', '3887 10.3 0.2 0.0 chrY 28675369 28675930 (33784099) + HSATI Satellite 6 568 (0) 36041']"

In [18]:
28673763-28673501

262

In [19]:
28675369-28673763

1606

In [8]:
T2T_df.loc['+_AluY_28675930-28676221']['Group']

"['1403 19.2 0.0 5.4 chrY 28675930 28676221 (33783808) + AluY SINE/Alu 13 289 (12) 36042 *', '382 19.3 8.9 3.0 chrY 28676223 28677768 (33782261) + (TA)n Simple_repeat 1 1634 (0) 36043']"

In [9]:
T2T_df.loc['+_AluY_28678343-28678634']['Group']

"['4026 9.5 0.0 0.0 chrY 28677776 28678343 (33781686) + HSATI Satellite 1 568 (0) 36044', '1465 18.1 0.0 5.4 chrY 28678343 28678634 (33781395) + AluY SINE/Alu 13 289 (12) 36045 *', '384 18.6 8.7 3.5 chrY 28678636 28680175 (33779854) + (TA)n Simple_repeat 1 1618 (0) 36046']"

In [ ]:
#Block 0
#chrY:28671944-28673763 (After Satellite in 77, includes next Alu)
#chrY:28675930-28677776 (Before Satellite (+_AluY_28678343-28678634), includes group before)

#chrY:28673501-28675930 (Alu up until before next Alu)
#chrY:28673763-28676221

## Inversion 2 Blocks 1 and 2

In [24]:
T2T_df[T2T_df['RepeatBlock']==1].loc['+_AluY_29237520-29237811']['Group']

"['3926 10.4 0.0 0.0 chrY 29236953 29237520 (33222509) + HSATI Satellite 1 568 (0) 36327', '1446 18.8 0.0 5.4 chrY 29237520 29237811 (33222218) + AluY SINE/Alu 13 289 (12) 36328 *', '385 18.6 9.1 3.0 chrY 29237814 29239364 (33220665) + (TA)n Simple_repeat 1 1645 (0) 36329']"

In [28]:
T2T_df[T2T_df['RepeatBlock']==2].head(2)

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End,SpaceBetween
Alu,,,,,,,,
C_AluY_29321438-29321729,2,1546,['172 24.4 4.1 3.9 chrY 29320751 29321435 (331...,1000-1999,#FF033E,29320751,29322296,81387
C_AluY_29323854-29324146,2,2410,['417 24.3 3.6 3.8 chrY 29322304 29323853 (331...,2000-2999,#FF033E,29322304,29324713,8


In [26]:
T2T_df[T2T_df['RepeatBlock']==2].loc['C_AluY_29321438-29321729']['Group']

"['172 24.4 4.1 3.9 chrY 29320751 29321435 (33138594) + (TATATAT)n Simple_repeat 1 686 (0) 36332', '1330 20.6 0.0 5.4 chrY 29321438 29321729 (33138300) C AluY SINE/Alu (12) 289 13 36333 *', '3940 10.4 0.0 0.0 chrY 29321729 29322296 (33137733) C HSATI Satellite (0) 568 1 36334']"

In [27]:
T2T_df[T2T_df['RepeatBlock']==2].loc['C_AluY_29323854-29324146']['Group']

"['417 24.3 3.6 3.8 chrY 29322304 29323853 (33136176) + (TATATAA)n Simple_repeat 1 1547 (0) 36335', '1392 19.1 0.0 5.4 chrY 29323854 29324146 (33135883) C AluY SINE/Alu (11) 290 13 36336 *', '3945 10.2 0.0 0.0 chrY 29324146 29324713 (33135316) C HSATI Satellite (0) 568 1 36337']"

In [ ]:
#Blocks1 n 2
#chrY:29237520-29239520
#chrY:29319438-29321729

## Inversion 3

In [29]:
T2T_df[T2T_df['RepeatBlock']==39]

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End,SpaceBetween
Alu,,,,,,,,
C_AluY_61054235-61054526,39,1609,['183 24.8 4.0 4.0 chrY 61053485 61054234 (140...,1000-1999,#FF033E,61053485,61055093,347729
C_AluY_61056657-61056947,39,2414,['373 25.8 4.0 3.4 chrY 61055101 61055762 (140...,2000-2999,#FF033E,61055101,61057514,8
C_AluY_61059078-61059369,39,2415,['377 25.8 3.4 3.8 chrY 61057522 61058183 (140...,2000-2999,#FF033E,61057522,61059936,8
C_AluY_61061497-61061788,39,2412,['395 24.3 4.2 3.5 chrY 61059944 61060604 (139...,2000-2999,#FF033E,61059944,61062355,8
C_AluY_61063917-61064208,39,2413,['428 23.8 3.7 3.7 chrY 61062363 61063916 (139...,2000-2999,#228B22,61062363,61064775,8
C_AluY_61066333-61066624,39,2409,['428 24.0 3.8 3.5 chrY 61064783 61066332 (139...,2000-2999,#FF033E,61064783,61067191,8
C_AluY_61068757-61069045,39,2414,['422 23.7 4.0 3.6 chrY 61067199 61068754 (139...,2000-2999,#FF033E,61067199,61069612,8
C_AluY_61071150-61071441,39,2394,['413 24.3 3.3 4.1 chrY 61069615 61071149 (138...,2000-2999,#FF033E,61069615,61072008,3
C_AluY_61073565-61073856,39,2408,['422 24.3 3.5 3.7 chrY 61072016 61073564 (138...,2000-2999,#FF033E,61072016,61074423,8


In [31]:
T2T_df.loc['C_AluY_61143977-61144266']['Group']

"['400 25.3 3.4 3.9 chrY 61142420 61143974 (1316055) + (TATATAA)n Simple_repeat 1 1548 (0) 62117', '1449 18.2 0.0 5.5 chrY 61143977 61144266 (1315763) C AluY SINE/Alu (14) 287 13 62118 *', '4016 9.5 0.0 0.0 chrY 61144266 61144833 (1315196) C HSATI Satellite (0) 568 1 62119']"

In [32]:
T2T_df.loc['+_AluY_61145504-61145782']['Group']

"['157 19.3 9.8 2.7 chrY 61144834 61145500 (1314529) + (AT)n Simple_repeat 1 713 (0) 62120', '1345 19.4 0.0 5.7 chrY 61145504 61145782 (1314247) + AluY SINE/Alu 26 289 (12) 62121', '391 19.1 8.8 3.0 chrY 61145784 61147334 (1312695) + (TA)n Simple_repeat 1 1638 (0) 62122']"

In [33]:
T2T_df.loc['+_AluY_61147909-61148200']['Group']

"['4011 9.5 0.0 0.0 chrY 61147342 61147909 (1312120) + HSATI Satellite 1 568 (0) 62123', '1424 18.5 0.0 5.4 chrY 61147909 61148200 (1311829) + AluY SINE/Alu 13 289 (12) 62124 *', '376 19.4 9.3 2.7 chrY 61148202 61149754 (1310275) + (TA)n Simple_repeat 1 1652 (0) 62125']"

In [ ]:
#Block 39
#chrY:61144266-61147334
#chrY:61144834-61147909

## T2T Breakpoint Inversion 4

In [39]:
T2T_df[T2T_df['RepeatBlock']==40].head(8)

,RepeatBlock,Lengths,Group,Block_Size,KMode_cluster,Start,End,SpaceBetween
Alu,,,,,,,,
+_AluY_61390448-61390739,40,2380,['3930 10.4 0.0 0.0 chrY 61389881 61390448 (10...,2000-2999,#FF033E,61389881,61392260,236093
+_AluY_61392835-61393126,40,3695,['4016 9.3 0.0 0.2 chrY 61392267 61392835 (106...,3000-3999,#FF033E,61392267,61395961,7
+_AluY_61396536-61396827,40,2407,['4005 9.7 0.0 0.0 chrY 61395969 61396536 (106...,2000-2999,#FF033E,61395969,61398375,8
+_AluY_61398949-61399240,40,2410,['3854 10.6 0.2 0.0 chrY 61398383 61398949 (10...,2000-2999,#228B22,61398383,61400792,8
+_AluY_61401367-61401658,40,2415,['3974 9.9 0.0 0.0 chrY 61400800 61401367 (105...,2000-2999,#F6C324,61400800,61403214,8
+_AluY_61403790-61404081,40,1751,['3884 10.4 0.0 0.2 chrY 61403222 61403790 (10...,1000-1999,#228B22,61403222,61404972,8
C_AluY_61407110-61407401,40,2409,['411 24.8 3.5 3.8 chrY 61405560 61407109 (105...,2000-2999,#FF033E,61405560,61407968,588
C_AluY_61409530-61409821,40,2413,['401 25.2 3.4 3.9 chrY 61407976 61409529 (105...,2000-2999,#228B22,61407976,61410388,8


In [35]:
T2T_df.loc['+_AluY_61401367-61401658']['Group']

"['3974 9.9 0.0 0.0 chrY 61400800 61401367 (1058662) + HSATI Satellite 1 568 (0) 62239', '1446 18.8 0.0 5.4 chrY 61401367 61401658 (1058371) + AluY SINE/Alu 13 289 (12) 62240 *', '418 24.6 3.9 3.4 chrY 61401659 61403214 (1056815) + (TTATATA)n Simple_repeat 1 1563 (0) 62241']"

In [36]:
T2T_df.loc['+_AluY_61403790-61404081']['Group']

"['3884 10.4 0.0 0.2 chrY 61403222 61403790 (1056239) + HSATI Satellite 1 568 (0) 62242', '1468 18.8 0.0 5.4 chrY 61403790 61404081 (1055948) + AluY SINE/Alu 13 289 (12) 62243 *', '231 18.6 8.1 3.7 chrY 61404083 61404972 (1055057) + (TA)n Simple_repeat 1 928 (0) 62244']"

In [37]:
T2T_df.loc['C_AluY_61407110-61407401']['Group']

"['411 24.8 3.5 3.8 chrY 61405560 61407109 (1052920) + (TATATAA)n Simple_repeat 1 1547 (0) 62246', '1439 18.5 0.0 5.4 chrY 61407110 61407401 (1052628) C AluY SINE/Alu (12) 289 13 62247 *', '3959 9.9 0.0 0.0 chrY 61407401 61407968 (1052061) C HSATI Satellite (0) 568 1 62248']"

In [ ]:
#block 40
#chrY:61403790-61405972
#chrY:61405560-61407401